In [ ]:
#
# This file contains the common functions and definitions used in the Egeria Hands on
# Lab Notebooks.
#

# These functions define the location and names of Coco Pharmaceutical's
# Open Metadata and Governance (OMAG) Server Platforms where the metadata servers
# and governance servers run.  The `os.environ.get` function tests for the presence
# of the environment variables that define the platform network addresses in the
# Docker and Kubernetes runtime environments.  If the environment variables are not
# present then the localhost defaults are used.

import os
import csv
import time

#
# Disable certificate checking for local development use with self-signed certificate 
#

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

import os
os.environ['CURL_CA_BUNDLE'] = ''

#
# This is the location of the file system that is used in the lab.  It defaults to a "data" subfolder in
# the platform's home folder.  Here you will find the runtime files of the platforms and the data files
# used in various labs.  The default value can be overridden using the "fileSystemRoot" environment variable.
#
fileSystemRoot = os.environ.get('fileSystemRoot', 'data')

#
# This is directory where the sample data from the Egeria distribution has been unpacked in.  The default value
# describes its location in an IntelliJ workspace.  The default value can be overridden using the "egeriaDistributionRoot" 
# environment variable.
#
egeriaSampleDataRoot = os.environ.get('egeriaSampleDataRoot', '.')

#
# Flag to enable debug, this cases extra information to be printed including rest calls request and response details
# Switching this flag to True produces a very large amount of output and is not recommended.
# A targeted use of this flag is recommended, set this before and reset this after the code you would like to produce debug
#
isDebug = False

In [ ]:
#
# Definitions of the Coco Pharmaceuticals Environment
#

# These are the three main platforms used to run Egeria's OMAG Servers
corePlatformURL      = os.environ.get('corePlatformURL', 'https://localhost:9443')
corePlatformName     = "Core Platform"

dataLakePlatformURL  = os.environ.get('dataLakePlatformURL', 'https://localhost:9444')
dataLakePlatformName = "Data Lake Platform"

devPlatformURL       = os.environ.get('devPlatformURL', 'https://localhost:9445')
devPlatformName      = "Dev Platform"


# The OMAG Server Platforms host one to many OMAG Servers.  An OMAG Server could be
# a metadata server or a specialized governance server.  Its behavior is determined
# by a configuration document that defines which OMAG services are activated.
# All OMAG Server Platforms support the administration commands to define a server's
# configuration document.  It is also possible to create configuration documents
# through admin calls to one OMAG Server Platform and then deploy them to the
# OMAG Server Platform where they are to run.  In the Egeria hands on lab, the
# OMAG Server configuration is created on the dev platform and deployed to the
# production platforms as needed.

adminPlatformURL = devPlatformURL

# Gary Geeke is the IT Administration Lead at Coco Pharmaceuticals.
# He does all of the configuration for the OMAG Servers. Other users are introduced and make
# calls to the server as required

adminUserId   = "garygeeke"
petersUserId  = "peterprofile"
erinsUserId   = "erinoverview"
calliesUserId = 'calliequartile'
faithsUserId =  'faithbroker'

# These are the names of the metadata servers used by Coco Pharmaceuticals.  Each metadata
# server runs as an OMAG Server on one of the OMAG Server Platforms

cocoMDS1PlatformURL  = dataLakePlatformURL
cocoMDS1PlatformName = dataLakePlatformName
cocoMDS1Name         = "cocoMDS1"

cocoMDS2PlatformURL  = corePlatformURL
cocoMDS2PlatformName = corePlatformName
cocoMDS2Name         = "cocoMDS2"

cocoMDS3PlatformURL  = corePlatformURL
cocoMDS3PlatformName = corePlatformName
cocoMDS3Name         = "cocoMDS3"

cocoMDS4PlatformURL  = dataLakePlatformURL
cocoMDS4PlatformName = dataLakePlatformName
cocoMDS4Name         = "cocoMDS4"

cocoMDS5PlatformURL  = corePlatformURL
cocoMDS5PlatformName = corePlatformName
cocoMDS5Name         = "cocoMDS5"

cocoMDS6PlatformURL  = corePlatformURL
cocoMDS6PlatformName = corePlatformName
cocoMDS6Name         = "cocoMDS6"

cocoMDSxPlatformURL  = devPlatformURL
cocoMDSxPlatformName = devPlatformName
cocoMDSxName         = "cocoMDSx"

cocoOLS1PlatformName = dataLakePlatformName
cocoOLS1PlafformURL  = dataLakePlatformURL
cocoOLS1Name         = "cocoOLS1"

# The open metadata servers are linked together through the following open metadata cohorts.
# The servers linked via a cohort can exchange open metadata either through federated
# queries or metadata replication.

cocoCohort = "cocoCohort"
devCohort  = "devCohort"
iotCohort  = "iotCohort"
ctsCohort  = "ctsCohort"

# This is the view server that runs the services that support Egeria's user interface.

cocoView1PlatformURL  = dataLakePlatformURL
cocoView1PlatformName = dataLakePlatformName
cocoView1Name = "cocoView1"

# These are the names of the governance servers used in Coco Pharmaceuticals' data lake.
# Each governance server runs as an OMAG Server on the Data Lake OMAG Server Platform.
# They also connect to a metadata server to retrieve their configuration and store their
# results.

governDL01PlatformURL    = dataLakePlatformURL
governDL01PlatformName   = dataLakePlatformName
governDL01Name           = "governDL01"
governDL01ServerType     = "Engine Host"
governDL01MDS            = "cocoMDS1"

exchangeDL01PlatformURL  = dataLakePlatformURL
exchangeDL01PlatformName = dataLakePlatformName
exchangeDL01Name         = "exchangeDL01"
exchangeDL01ServerType   = "Integration Daemon"
exchangeDL01MDS          = "cocoMDS1"

monitorGov01PlatformURL  = dataLakePlatformURL
monitorGov01PlatformName = dataLakePlatformName
monitorGov01Name         = "monitorGov01"
monitorGov01ServerType   = "Integration Daemon"
monitorGov01MDS          = "cocoMDS1"

monitorDev01PlatformURL  = devPlatformURL
monitorDev01PlatformName = devPlatformName
monitorDev01Name         = "monitorDev01"
monitorDev01ServerType   = "Integration Daemon"
monitorDev01MDS          = "cocoMDSx"


In [ ]:
#
# Common processing of REST API errors.
#

import requests
import pprint
import json


def printResponse(response):
    prettyResponse = json.dumps(response.json(), indent=4)
    print(" ")
    print("Response: ")
    print(prettyResponse)
    print(" ")
    
def printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response):
    if response.status_code == 200:
        relatedHTTPCode = response.json().get('relatedHTTPCode')
        if relatedHTTPCode == 200:
            print("Unexpected response from server " + serverName)
            printResponse(response)
        else:
            exceptionErrorMessage = response.json().get('exceptionErrorMessage')
            exceptionSystemAction = response.json().get('exceptionSystemAction')
            exceptionUserAction   = response.json().get('exceptionUserAction')
            if exceptionErrorMessage != None:
                print(exceptionErrorMessage)
                print(" * " + exceptionSystemAction)
                print(" * " + exceptionUserAction)
            else:
                print("Unexpected response from server " + serverName)
                printResponse(response)
    else:
        print("Unexpected response from server platform " + serverPlatformName + " at " + serverPlatformURL)
        printResponse(response)

#
# Rest calls, these functions issue rest calls and print debug if required.
# 
def issuePostNoBody(url):
    if (isDebug):
        printRestRequest("POST " + url)
    jsonHeader = {'content-type':'application/json'}
    response=requests.post(url, json=None, headers=None, verify=False)
    if (isDebug):
        printRestResponse(response) 
    return response

def issuePost(url, body):
    if (isDebug):
        printRestRequest("POST " + url)
        printRestRequestBody(body)
    jsonHeader = {'content-type':'application/json'}
    response=requests.post(url, json=body, headers=jsonHeader, verify=False)
    if (isDebug):
        printRestResponse(response) 
    return response

def issueDataPost(url, body):
    if (isDebug):
        printRestRequest("POST " + url)
        printRestRequestBody(body)
    jsonHeader = {'content-type':'text/plain'}
    response=requests.post(url, data=body, verify=False)
    if (isDebug):
        printRestResponse(response) 
    return response

def issueDelete(url):
    if (isDebug):
        printRestRequest("DELETE " + url)
    response=requests.delete(url, json=None, headers=None, verify=False)
    if (isDebug):
        printRestResponse(response) 
    return response

def issuePut(url, body):
    if (isDebug):
        printRestRequest("PUT " + url)
        printRestRequestBody(body)
    jsonHeader = {'content-type':'application/json'}
    response=requests.put(url, json=body, headers=jsonHeader, verify=False)
    if (isDebug):
        printRestResponse(response) 
    return response

def issueGet(url):
    if (isDebug):
        printRestRequest("GET " + url)
    jsonHeader = {'content-type':'application/json'}
    response=requests.get(url, headers=jsonHeader, verify=False)
    if (isDebug):
        printRestResponse(response) 
    return response

def printRestRequest(url):
    print (" ")
    print (url)
    
def printRestRequestBody(body):   
    prettyBody = json.dumps(body, indent=4)
    print (prettyBody)
    print (" ")
    
def printRestResponse(response): 
    print ("Returns:")
    prettyResponse = json.dumps(response.json(), indent=4)
    print (prettyResponse)
    print (" ")

def processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response):
    if response.status_code != 200:
        printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    else:
        relatedHTTPCode = response.json().get('relatedHTTPCode')
        if relatedHTTPCode != 200:
             printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    return []

def postAndPrintResult(url, json=None, headers=None):
    print ("   ...... (POST", url, ")")
    response = requests.post(url, json=json, headers=headers, verify=False)
    print ("   ...... Response: ", response.json())


In [ ]:

#
# Administration services
#

#
# OMAG Server configuration functions.  These functions add definitions to an OMAG server's configuration document
#
def configurePlatformURL(adminPlatformURL, adminUserId, serverName, serverPlatform):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the platform the server will run on...")
    url = adminCommandURLRoot + serverName + '/server-url-root?url=' + serverPlatform
    issuePostNoBody(url)

def configureMaxPageSize(adminPlatformURL, adminUserId, serverName, maxPageSize):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the maximum page size...")
    url = adminCommandURLRoot + serverName + '/max-page-size?limit=' + maxPageSize
    issuePostNoBody(url)

def configureServerType(adminPlatformURL, adminUserId, serverName, serverType):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the server's type...")
    url = adminCommandURLRoot + serverName + '/server-type?typeName=' + serverType
    issuePostNoBody(url)
    
def clearServerType(adminPlatformURL, adminUserId, serverName):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... clearing the server's type...")
    url = adminCommandURLRoot + serverName + '/server-type?typeName='
    issuePostNoBody(url)

def configureOwningOrganization(adminPlatformURL, adminUserId, serverName, organizationName):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the server's owning organization...")
    url = adminCommandURLRoot + serverName + '/organization-name?name=' + organizationName
    issuePostNoBody(url)

def configureUserId(adminPlatformURL, adminUserId, serverName, userId):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the server's userId...")
    url = adminCommandURLRoot + serverName + '/server-user-id?id=' + userId
    issuePostNoBody(url)

def configurePassword(adminPlatformURL, adminUserId, serverName, password):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the server's password (optional)...")
    url = adminCommandURLRoot + serverName + '/server-user-password?password=' + password
    issuePostNoBody(url)

def configureSecurityConnection(adminPlatformURL, adminUserId, serverName, securityBody):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the server's security connection...")
    url = adminCommandURLRoot + serverName + '/security/connection'
    issuePost(url, securityBody)

def configureDefaultAuditLog(adminPlatformURL, adminUserId, serverName):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the default audit log...")
    url = adminCommandURLRoot + serverName + '/audit-log-destinations/default'
    issuePostNoBody(url)
    
#
# These commands are for metadata servers, metadata access points and repository proxies

def configurePluginMetadataRepository(adminPlatformURL, adminUserId, serverName, repositoryConnectionBody):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the plugin repository...")
    url = adminCommandURLRoot + serverName + '/local-repository/mode/plugin-repository/connection'
    issuePost(url,repositoryConnectionBody)
    
def configureBuiltinMetadataRepository(adminPlatformURL, adminUserId, serverName, repositoryType):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the builtin metadata repository...")
    url = adminCommandURLRoot + serverName + '/local-repository/mode/' + repositoryType
    issuePostNoBody(url)
    
def configureMetadataRepository(adminPlatformURL, adminUserId, serverName, repositoryType):
    # xtdb isn't internal, but we'll add support in these common functions
    if (repositoryType == 'xtdb-inmemory'):
        repositoryConnectionBody = {
            "class": "Connection",
            "connectorType": {
                "class": "ConnectorType",
                "connectorProviderClassName": "org.odpi.egeria.connectors.juxt.xtdb.repositoryconnector.XtdbOMRSRepositoryConnectorProvider"
            }
        }
        configurePluginMetadataRepository(adminPlatformURL, adminUserId, serverName, repositoryConnectionBody)
    else:
        configureBuiltinMetadataRepository(adminPlatformURL, adminUserId, serverName, repositoryType)
    
def configureRepositoryProxyDetails(adminPlatformURL, adminUserId, serverName, connectorProvider):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the repository proxy...")
    url = adminCommandURLRoot + serverName + '/local-repository/mode/repository-proxy/details?connectorProvider=' + connectorProvider
    issuePostNoBody(url)

def configureDescriptiveName(adminPlatformURL, adminUserId, serverName, collectionName):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the metadata collection name of the metadata stored in this server...")
    url = adminCommandURLRoot + serverName + '/local-repository/metadata-collection-name/' + collectionName
    issuePostNoBody(url)
    
def configureMetadataCollectionId(adminPlatformURL, adminUserId, serverName, collectionId):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the metadata collection id of the metadata stored in this server...")
    url = adminCommandURLRoot + serverName + '/local-repository/metadata-collection-id'
    issueDataPost(url, collectionId)
    
def addStartupArchive(adminPlatformURL, adminUserId, serverName, archiveFileName):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... adding start up archive " + archiveFileName + " to server ...")
    url = adminCommandURLRoot + serverName + '/open-metadata-archives/file'
    issueDataPost(url, archiveFileName)
    
def removeAllStartupArchive(adminPlatformURL, adminUserId, serverName):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... removing all start up archives from server ...")
    url = adminCommandURLRoot + serverName + '/open-metadata-archives'
    issueDelete(url)

def configureEventBus(adminPlatformURL, adminUserId, serverName, busBody):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the event bus for this server...")
    url = adminCommandURLRoot + serverName + '/event-bus'
    issuePost(url, busBody)

def configureCohortMembership(adminPlatformURL, adminUserId, serverName, cohortName):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the membership of the cohort...")
    url = adminCommandURLRoot + serverName + '/cohorts/' + cohortName
    issuePostNoBody(url)
    
def deleteCohortMembership(adminPlatformURL, adminUserId, serverName, cohortName):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... removing the membership of the cohort...")
    url = adminCommandURLRoot + serverName + '/cohorts/' + cohortName
    issueDelete(url)

def configureAccessService(adminPlatformURL, adminUserId, serverName, accessService, accessServiceOptions):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the " + accessService + " access service for this server...")
    url = adminCommandURLRoot + serverName + '/access-services/' + accessService
    issuePost(url, accessServiceOptions)
    
#
# The commands below are for View Servers only
#
def configureGovernanceSolutionViewService(adminPlatformURL, adminUserId, viewServerName, viewService, remotePlatformURL,remoteServerName):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the " + viewService + " Governance Solution View Service for this server...")
    url = adminCommandURLRoot + viewServerName + '/view-services/' + viewService
    viewBody = {
        "class": "ViewServiceConfig",
        "omagserverPlatformRootURL": remotePlatformURL,
        "omagserverName" : remoteServerName
    }
    issuePost(url, viewBody)
    
def configureIntegrationViewService(adminPlatformURL, adminUserId, viewServerName, viewService, configBody):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the " + viewService + " Integration View Service for this server...")
    url = adminCommandURLRoot + viewServerName + '/view-services/' + viewService
    issuePost(url, configBody)

#
# The commands below are for Integration Daemon OMAG Servers only
#
def configureIntegrationService(adminPlatformURL, adminUserId, daemonServerName, mdrServerName, mdrServerPlatform, integrationServiceURLMarker, integrationServiceOptions, connectorConfigs):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the " + integrationServiceURLMarker + " integration service in " + daemonServerName + " integration daemon ...")
    requestBody = {
        "class": "IntegrationServiceRequestBody",
        "omagserverPlatformRootURL": mdrServerPlatform,
        "omagserverName" : mdrServerName,
        "integrationServiceOptions" : integrationServiceOptions,
        "integrationConnectorConfigs" : connectorConfigs
    }
    url = adminCommandURLRoot + daemonServerName + '/integration-services/' + integrationServiceURLMarker
    issuePost(url, requestBody)

#
# The commands below are for Engine Host OMAG Servers only
#
def configureEngineDefinitionServices(adminPlatformURL, adminUserId, engineServerName, mdrServerName, mdrServerPlatform):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the " + engineServerName + " engine definition services...")
    requestBody = {
        "class": "OMAGServerClientConfig",
        "omagserverPlatformRootURL": mdrServerPlatform,
        "omagserverName" : mdrServerName
    }
    url = adminCommandURLRoot + engineServerName + '/engine-definitions/client-config'
    issuePost(url, requestBody)

def configureGovernanceEngineService(adminPlatformURL, adminUserId, engineServerName, mdrServerName, mdrServerPlatform, engineServiceURLMarker, governanceEngines):
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    print ("   ... configuring the " + engineServiceURLMarker + " engine service in " + engineServerName + " engine host server ...")
    requestBody = {
        "class": "EngineServiceRequestBody",
        "omagserverPlatformRootURL": mdrServerPlatform,
        "omagserverName" : mdrServerName,
        "engines" : governanceEngines
    }
    url = adminCommandURLRoot + engineServerName + '/engine-services/' + engineServiceURLMarker
    issuePost(url, requestBody)

In [ ]:
# Server configuration can be deployed from one OMAG Server Platform to another.
# This enables a server configuration to be tested and then deployed in production.

def deployServerToPlatform(adminPlatformURL, adminUserId, serverName, platformURL):
    print("   ... deploying", serverName, "to the", platformURL, "platform...")
    adminCommandURLRoot = adminPlatformURL + '/open-metadata/admin-services/users/' + adminUserId + '/servers/'
    url = adminCommandURLRoot + serverName + '/configuration/deploy'
    platformTarget = {
        "class": "URLRequestBody",
        "urlRoot": platformURL
    }
    jsonContentHeader = {'content-type':'application/json'}
    postAndPrintResult(url, json=platformTarget, headers=jsonContentHeader)


In [ ]:
# The OMAG Server Platform is a single executable (application) that can be started
# from the command line or a script or as part of a pre-built container environment
# such as Kubernetes.  The function below checks that a specific
# server platform is running.

def checkServerPlatform(serverPlatformName, serverPlatformURL):
    try:
        isPlatformActiveURL = serverPlatformURL + "/open-metadata/platform-services/users/" + adminUserId + "/server-platform/origin"
        response = requests.get(isPlatformActiveURL, verify=False)
        if response.status_code == 200:
            print("   ", serverPlatformName, "is active")
            return True
        else:
            print("   ", serverPlatformName, "is not an OMAG Platform")
            return False
    except Exception as error:
        print("Exception: %s" % error)
        print("   ", serverPlatformName, "is down - start it before proceeding")
        return False


# The OMAG Server Platform has the implementation of the open metadata and governance (OMAG) services.  These are
# divided into three groups: Common Services, Access Services and Governance Services.

def printServiceDescriptions(serverPlatformName, serviceGroupName, services):
    print(serviceGroupName, "for", serverPlatformName)
    for x in range(len(services)):
        serviceName = services[x].get('serviceName')
        serviceDescription = services[x].get('serviceDescription')
        serviceWiki = services[x].get('serviceWiki')    
        print (" * ", serviceName)
        print ("     ", serviceDescription)
        print ("     ", serviceWiki)
    print (" ")
    
def getServerPlatformServices(serverPlatformName, serverPlatformURL):
    getOMAGServicesURL = serverPlatformURL + "/open-metadata/platform-services/users/" + adminUserId + "/server-platform/registered-services"
    response = issueGet(getOMAGServicesURL + "/framework-services")
    if response.status_code == 200:
        printServiceDescriptions(serverPlatformName, "Common services", response.json().get('services'))
    response = issueGet(getOMAGServicesURL + "/access-services")
    if response.status_code == 200:
        printServiceDescriptions(serverPlatformName, "Access services", response.json().get('services'))
    response = issueGet(getOMAGServicesURL + "/engine-services")
    if response.status_code == 200:
        printServiceDescriptions(serverPlatformName, "Engine services", response.json().get('services'))
    response = issueGet(getOMAGServicesURL + "/integration-services")
    if response.status_code == 200:
        printServiceDescriptions(serverPlatformName, "Integration services", response.json().get('services'))
    response = issueGet(getOMAGServicesURL + "/view-services")
    if response.status_code == 200:
        printServiceDescriptions(serverPlatformName, "View services", response.json().get('services'))
    response = issueGet(getOMAGServicesURL + "/governance-services")
    if response.status_code == 200:
        printServiceDescriptions(serverPlatformName, "Governance services", response.json().get('services'))
        
def getAccessServices(serverPlatformName, serverPlatformURL):
    getOMAGServicesURL = serverPlatformURL + "/open-metadata/platform-services/users/" + adminUserId + "/server-platform/registered-services"
    response = issueGet(getOMAGServicesURL + "/access-services")
    if response.status_code == 200:
        printServiceDescriptions(serverPlatformName, "Access services", response.json().get('services'))
    
def getEngineServices(serverPlatformName, serverPlatformURL):
    getOMAGServicesURL = serverPlatformURL + "/open-metadata/platform-services/users/" + adminUserId + "/server-platform/registered-services"
    response =issueGet(getOMAGServicesURL + "/engine-services")
    if response.status_code == 200:
        printServiceDescriptions(serverPlatformName, "Engine services", response.json().get('services'))
              
def getIntegrationServices(serverPlatformName, serverPlatformURL):
    getOMAGServicesURL = serverPlatformURL + "/open-metadata/platform-services/users/" + adminUserId + "/server-platform/registered-services"
    response = issueGet(getOMAGServicesURL + "/integration-services")
    if response.status_code == 200:
        printServiceDescriptions(serverPlatformName, "Integration services", response.json().get('services'))
        
def getViewServices(serverPlatformName, serverPlatformURL):
    getOMAGServicesURL = serverPlatformURL + "/open-metadata/platform-services/users/" + adminUserId + "/server-platform/registered-services"
    response = issueGet(getOMAGServicesURL + "/view-services")
    if response.status_code == 200:
        printServiceDescriptions(serverPlatformName, "View services", response.json().get('services'))        

def getConnectorType(serverPlatformName, serverPlatformURL, connectorProviderClassName):
    print (" ")
    platformServicesURLRoot = serverPlatformURL + "/open-metadata/platform-services/users/" + adminUserId + "/server-platform"
    url = platformServicesURLRoot + '/connector-types/' + connectorProviderClassName
    response = requests.get(url, verify=False)
    if response.status_code == 200:
        connectorType = response.json().get('connectorType')
        if connectorType:
            print ("Connector Type: " + connectorType.get('displayName'))
            print ("    qualifiedName: " + connectorType.get('qualifiedName'))
            print ("    description: " + connectorType.get('description'))
            print ("    connectorProviderClassName: " + connectorType.get('connectorProviderClassName'))
            if connectorType.get('supportedAssetTypeName'):
                print ("    supportedAssetTypeName: " + connectorType.get('supportedAssetTypeName'))
            if connectorType.get('expectedDataFormat'):
                print ("    expectedDataFormat: " + connectorType.get('expectedDataFormat'))
            recognizedConfigurationProperties = connectorType.get('recognizedConfigurationProperties')
            if recognizedConfigurationProperties:
                print ("    recognizedConfigurationProperties:")
                for x in range(len(recognizedConfigurationProperties)):
                    print ("        " + recognizedConfigurationProperties[x])
            if connectorType.get('connectorInterfaceLanguage'):
                print ("    connectorInterfaceLanguage: " + connectorType.get('connectorInterfaceLanguage'))
            if connectorType.get('connectorFrameworkName'):
                print ("    connectorFrameworkName: " + connectorType.get('connectorFrameworkName'))
            connectorInterfaces = connectorType.get('connectorInterfaces')
            if connectorInterfaces:
                print ("    connectorInterfaces:")
                for x in range(len(connectorInterfaces)):
                    print ("        " + connectorInterfaces[x])
            if connectorType.get('targetTechnologySource'):
                print ("    targetTechnologySource: " + connectorType.get('targetTechnologySource'))
            if connectorType.get('targetTechnologyName'):
                print ("    targetTechnologyName: " + connectorType.get('targetTechnologyName'))
        else:
            prettyResponse = json.dumps(response.json(), indent=4)
            print ("Response: ")
            print (prettyResponse)
    else:
        prettyResponse = json.dumps(response.json(), indent=4)
        print ("Response: ")
        print (prettyResponse)
    print (" ")    


def queryKnownServers(serverPlatformName, serverPlatformURL):
    print (" ")
    print ("Querying the known servers on platform: " + serverPlatformName + " ...")
    platformServicesURLRoot = serverPlatformURL + "/open-metadata/platform-services/users/" + adminUserId + "/server-platform"
    url = platformServicesURLRoot + '/servers'
    print ("GET " + url)
    response = requests.get(url, verify=False)
    prettyResponse = json.dumps(response.json(), indent=4)
    print ("Response: ")
    print (prettyResponse)
    print (" ")

def queryActiveServers(serverPlatformName, serverPlatformURL):
    print (" ")
    print ("Active servers on platform: " + serverPlatformName + " ...")
    platformServicesURLRoot = serverPlatformURL + "/open-metadata/platform-services/users/" + adminUserId + "/server-platform"
    url = platformServicesURLRoot + '/servers/active'
    response = issueGet(url)
    if response.status_code == 200:
        servers = response.json().get('serverList')
        if servers is not None:
            print("  ", servers)
    else:
        prettyResponse = json.dumps(response.json(), indent=4)
        print ("Response: ")
        print (prettyResponse)
        print (" ")


In [ ]:

# Each server is configured to define which services should be active.  This configuration results in
# the creation of a configuration document.  This document is read when the server is started and
# drives the initialization of the services.

def checkServerConfigured(serverName, serverPlatformName, serverPlatformURL):
    isServerKnownOnPlatform = serverPlatformURL + "/open-metadata/admin-services/users/" + adminUserId + "/servers/" + serverName + "/configuration"
    response = issueGet(isServerKnownOnPlatform)
    if response.status_code == 200:
        serverConfig=response.json().get('omagserverConfig')
        auditTrail = serverConfig.get('auditTrail')
        if auditTrail is not None:
            print("           ...", serverName, "is configured")
            return True
        else:
            print("           ...", serverName, "needs configuring")
    else:
        print("   ...", serverPlatformName, "at", serverPlatformURL, "is down - unable to check server configuration")
        return False


# The OMAG Server Platform supports a call to return if a server is active.

def checkServerActive(serverName, serverPlatformName, serverPlatformURL):
    isServerActiveOnPlatform = serverPlatformURL + "/open-metadata/platform-services/users/" + adminUserId + "/server-platform/servers/" + serverName + "/status"
    response = issueGet(isServerActiveOnPlatform)
    if response.status_code == 200:
        relatedHTTPCode = response.json().get('relatedHTTPCode')
        if relatedHTTPCode == 200:
            serverStatus = response.json().get('active')
            if serverStatus == True:
                print("           ...", serverName, "is active - ready to begin")
            else:
                print("           ...", serverName, "is down - needs to be started")
            return serverStatus
        else:
            print("           ...", serverName, "is not known - needs to be started")
        return False
    else:
        print("   ...", serverPlatformName, "at", serverPlatformURL, "is down - unable to check server configuration")
        return False

    
# This is the call to start a server on a specific platform.  Once the server is running it is possible to
# make use of the open metadata and governance services that have been activated in the server.

def activateServerOnPlatform(serverName, serverPlatformName, serverPlatformURL):
    print ("                Starting server " + serverName + " ...")
    activateServerURL = serverPlatformURL + "/open-metadata/admin-services/users/" + adminUserId + '/servers/' + serverName + "/instance"
    response = issuePost(activateServerURL, {})
    if response.status_code == 200:
        relatedHTTPCode = response.json().get('relatedHTTPCode')
        if relatedHTTPCode == 200:
            return True
        else:
            errorMessage = response.json().get('exceptionErrorMessage')
            print ("                   ..." + serverName + " failed to start")
            print ("                       error message is: " + errorMessage)
            return False
                
    else:
        print ("                   ..." + serverName + " failed to start")
        return False


# Once a server is active, it is possible to query the services that are active.

def getServerServices(serverName, serverPlatformName, serverPlatformURL):
    getServerActiveServicesURL = serverPlatformURL + "/open-metadata/platform-services/users/" + adminUserId + "/server-platform/servers/" + serverName + "/services"
    print ("Services for server:", serverName)
    response = issueGet(getServerActiveServicesURL)
    if response.status_code == 200:
        serviceList = response.json().get('serverServicesList')
        for x in range(len(serviceList)):
            print (" * ", serviceList[x])
    else:
        print (response)

# This function checks whether a server is active and starts it if it is down.

def activateServerIfDown(serverName, serverPlatformName, serverPlatformURL):
    print("        Checking OMAG Server " + serverName)
    configured = checkServerConfigured(serverName, serverPlatformName, serverPlatformURL)
    if configured == True:
        active = checkServerActive(serverName, serverPlatformName, serverPlatformURL)
        if active == False:
            activateServerOnPlatform(serverName, serverPlatformName, serverPlatformURL)

# This function checks the platform is running and ensures the servers are started on it.
# It requests user action if either the platform is not running or the servers are not configured.
# Otherwise it should return with all of the servers running.

def activatePlatform(serverPlatformName, serverPlatformURL, hostedServerNames):
    available = checkServerPlatform(serverPlatformName, serverPlatformURL)
    if available == True:
        for x in range(len(hostedServerNames)):
            activateServerIfDown(hostedServerNames[x], serverPlatformName, serverPlatformURL)

# This function checks the platform is running and (re)starts each of the requested servers.

def reActivatePlatform(serverPlatformName, serverPlatformURL, hostedServerNames):
    available = checkServerPlatform(serverPlatformName, serverPlatformURL)
    if available == True:
        for x in range(len(hostedServerNames)):
            activateServerOnPlatform(hostedServerNames[x], serverPlatformName, serverPlatformURL)



In [ ]:
#
# Repository Services
#

# Understanding Cohorts
#
# The metadata servers, metadata access points, repository proxies and the CTS are linked together through open metadata repository cohorts.
# The servers linked via a cohort can exchange open metadata either through federated queries or metadata replication.

def queryServerCohorts(serverName, serverPlatformName, serverPlatformURL):
    cohortNames = []
    try:
        metadataHighwayServicesURLcore =  '/servers/' + serverName + '/open-metadata/repository-services/users/' + adminUserId + '/metadata-highway'
        url = serverPlatformURL + metadataHighwayServicesURLcore + '/cohort-descriptions'
        response = requests.get(url, verify=False)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                cohorts=response.json().get('cohorts')
                if cohorts != None:
                    for x in range(len(cohorts)):
                        cohortName = cohorts[x].get('cohortName')
                        cohortNames.append(cohortName)
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except Exception as error:
        print("Exception: %s" % error)
        print("Platform " + serverPlatformURL + " is returning an error")
    return cohortNames


def printServerCohortsStatus(serverName, serverPlatformName, serverPlatformURL):
    try:
        metadataHighwayServicesURLcore =  '/servers/' + serverName + '/open-metadata/repository-services/users/' + adminUserId + '/metadata-highway'
        url = serverPlatformURL + metadataHighwayServicesURLcore + '/cohort-descriptions'
        response = issueGet(url)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                cohorts=response.json().get('cohorts')
                if cohorts == None:
                    print("Server " + serverName + " is not connected to any cohorts")
                else:
                    print("Server " + serverName + " is connected to the following cohorts:")
                    for x in range(len(cohorts)):
                        cohortName = cohorts[x].get('cohortName')
                        connectionStatus = cohorts[x].get('connectionStatus')
                        print (" * " + cohortName + " [" + connectionStatus + "]")
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except Exception as error:
        print("Exception: %s" % error)
        print("Platform " + serverPlatformURL + " is returning an error")


def printCohortMember(cohortMember, localRegistration):
    serverName = cohortMember.get('serverName')
    serverType = cohortMember.get('serverType')
    metadataCollectionId = cohortMember.get('metadataCollectionId')
    metadataCollectionName = cohortMember.get('metadataCollectionName')
    registrationTime = cohortMember.get('registrationTime')
    if localRegistration == True:
        print("Registration details for local " + serverType + " " + serverName)
    else:    
        print("Registration details for remote " + serverType + " " + serverName)
    if (metadataCollectionId != None):
        print(" * Metadata collection id:   " + metadataCollectionId)
    if (metadataCollectionName != None):
        print(" * Metadata collection name: " + metadataCollectionName)
    if (registrationTime != None):
        print(" * Registration time:        " + registrationTime)
    repositoryConnection = cohortMember.get('repositoryConnection')
    if repositoryConnection != None:
        endpoint = repositoryConnection.get('endpoint')
        if endpoint != None:
            address = endpoint.get('address')
            if address != None:
                print(" * URL for metadata queries: " + address)
            else:
                print(" * URL for metadata queries: null")
        else:
            print(" * URL for metadata queries: no endpoint")
    else:
        print(" * URL for metadata queries: not supported")


def printLocalRegistration(serverName, serverPlatformName, serverPlatformURL):
    try:
        metadataHighwayServicesURLcore =  '/servers/' + serverName + '/open-metadata/repository-services/users/' + adminUserId + '/metadata-highway'
        url = serverPlatformURL + metadataHighwayServicesURLcore + '/local-registration'
        response = issueGet(url)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                cohortMember = response.json().get('cohortMember')
                printCohortMember(cohortMember, True)
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except Exception as error:
        print("Exception: %s" % error)
        print("Platform " + serverPlatformURL + " is returning an error")


def printLocalRegistrationForCohort(serverName, cohortName, serverPlatformName, serverPlatformURL):
    try:
        metadataHighwayServicesURLcore =  '/servers/' + serverName + '/open-metadata/repository-services/users/' + adminUserId + '/metadata-highway'
        url = serverPlatformURL + metadataHighwayServicesURLcore + '/cohorts/' + cohortName + '/local-registration'
        response = issueGet(url)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                cohortMember = response.json().get('cohortMember')
                printCohortMember(cohortMember, True)
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except Exception as error:
        print("Exception: %s" % error)
        print("Platform " + serverPlatformURL + " is returning an error")


def printRemoteRegistrations(serverName, cohortName, serverPlatformName, serverPlatformURL):
    try:
        metadataHighwayServicesURLcore = '/servers/' + serverName + '/open-metadata/repository-services/users/' + adminUserId + '/metadata-highway'
        url = serverPlatformURL + metadataHighwayServicesURLcore + '/cohorts/' + cohortName + '/remote-members'
        response = issueGet(url)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                cohortMembers = response.json().get('cohortMembers')
                if cohortMembers != None:
                    for x in range(len(cohortMembers)):
                        printCohortMember(cohortMembers[x], False)
                else:
                    print("No remote members")
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except Exception as error:
        print("Exception: %s" % error)
        print("Platform " + serverPlatformURL + " is returning an error")
    

def printServerCohorts(serverName, serverPlatformName, serverPlatformURL):
    print("Reviewing the cohort registry for server: " + serverName)
    print(" ")
    printLocalRegistration(serverName, serverPlatformName, serverPlatformURL)
    print(" ")
    cohorts = queryServerCohorts(serverName, serverPlatformName, serverPlatformURL)
    for x in range(len(cohorts)):
        print("Cohort " + cohorts[x] + " member details")
        printLocalRegistrationForCohort(serverName, cohorts[x], serverPlatformName, serverPlatformURL)
        printRemoteRegistrations(serverName, cohorts[x], serverPlatformName, serverPlatformURL)
        print(" ")

def unregisterFromCohort(serverName, cohortName, serverPlatformName, serverPlatformURL):
    metadataHighwayServicesURLcore = '/servers/' + serverName + '/open-metadata/repository-services/users/' + adminUserId + '/metadata-highway'
    url = serverPlatformURL + metadataHighwayServicesURLcore + '/cohorts/' + cohortName + '/unregister'
    response = issueGet(url)


In [ ]:
#
# OMAS
# Working with asset elements - this set of functions displays assets returned from the open metadata repositories.
# 

def printAssetSummary(asset):
    elementHeader = asset.get('elementHeader')
    elementType = elementHeader.get('type')
    assetTypeName = elementType.get('typeName')
    requestType = elementHeader.get('guid')
    assetProperties = asset.get('assetProperties')
    assetQualifiedName = assetProperties.get('qualifiedName')
    print (assetQualifiedName)
    print(assetTypeName + " \t| " + requestType + " | " + assetQualifiedName)
    
def printAssetDetail(asset):
    elementHeader = asset.get('elementHeader')
    elementType = elementHeader.get('type')
    assetTypeName = elementType.get('typeName')
    assetSuperTypes = elementType.get('superTypeNames')
    assetGUID = elementHeader.get('guid')
    assetProperties = asset.get('assetProperties')
    assetQualifiedName = assetProperties.get('qualifiedName')
    assetDisplayName = assetProperties.get('displayName')
    assetVersion = assetProperties.get('versionIdentifier')
    extendedProperties = assetProperties.get('extendedProperties')
    if extendedProperties:
        assetPathName = extendedProperties.get('pathName')
    else:
        assetPathName = None
    assetDescription = assetProperties.get('description')
    assetOwner = assetProperties.get('owner')
    assetOrigin = assetProperties.get('otherOriginValues')
    assetOwnerType = assetProperties.get('ownerTypeName')
    assetZones = assetProperties.get('zoneMembership')
    if not assetGUID:
        assetGUID = "<null>"
    if not assetDisplayName:
        assetDisplayName = "<none>"
    else:
        textWidth = len(assetDisplayName)
        if textWidth > 20:
            assetDisplayName = "..." + assetDisplayName[textWidth-20:textWidth]
    print(assetDisplayName + " [" + assetGUID + "]")
    if not assetQualifiedName:
        assetQualifiedName = "<null>"
    print("  qualifiedName:     " + assetQualifiedName)
    if assetPathName:
        print("  pathName:          " + assetPathName)
    if assetVersion:
        print("  versionIdentifier: " + assetVersion)
    if assetDescription:
        print("  description:       " + assetDescription)
    print("  type:              " + assetTypeName + ' [%s]' % ', '.join(map(str, assetSuperTypes)))
    if assetOwner:
        print("  owner:             " + assetOwner + " [" + assetOwnerType + "]")
    if assetOrigin:
        contact = assetOrigin.get('contact')
        dept = assetOrigin.get('originatingDept')
        org = assetOrigin.get('originatingOrganization')
        print("  origin:            contact=" + contact + ", dept=" + dept + ", org=" + org)
    if assetZones:
        print("  zones:             " + '%s' % ', '.join(map(str, assetZones)))
    

def printAssetListSummary(assets):
    print(" ")
    for x in range(len(assets)):
        printAssetSummary(assets[x])

def printAssetListDetail(assets):
    print("\n--------------------------------------\n")
    for x in range(len(assets)):
        printAssetDetail(assets[x])
        print("\n--------------------------------------\n")


def printGUIDList(guids):
    if guids == None:
        print("No assets created")
    else:
        prettyGUIDs = json.dumps(guids, indent=4)
        print(prettyGUIDs)

def getLastGUID(guids):
    if guids == None:
        return "<unknown>"
    else:
        for guid in guids:
            returnGUID = guid
        return returnGUID

In [ ]:
#
# OCF Common services
# Working with assets - this set of functions displays assets returned from the open metadata repositories.
# 


def getAssetUniverse(serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, assetGUID):
    connectedAssetURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/framework-services/' + serviceURLMarker + '/connected-asset/users/' + userId 
    getAsset = connectedAssetURL + '/assets/' + assetGUID
    response = issueGet(getAsset)
    asset = response.json().get('asset')
    if asset:
        return response.json()
    else:
        print ("No Asset returned")
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response) 

def getRelatedAssets(serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, assetGUID):
    connectedAssetURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/framework-services/' + serviceURLMarker + '/connected-asset/users/' + userId 
    getRelatedAsset = connectedAssetURL + '/assets/' + assetGUID + '/related-assets?elementStart=0&maxElements=50'
    response = issueGet(getRelatedAsset)
    if response.status_code == 200:
        relatedHTTPCode = response.json().get('relatedHTTPCode')
        if relatedHTTPCode == 200:
            return response.json().get('list')            
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    else:
        printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)

def getComments(serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, assetGUID):
    connectedAssetURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/framework-services/' + serviceURLMarker + '/connected-asset/users/' + userId 
    commentQuery = connectedAssetURL + '/assets/' + assetGUID + '/comments?elementStart=0&maxElements=50'
    response = issueGet(commentQuery)
    responseObjects = response.json().get('list')
    if responseObjects:
        return responseObjects
    else:
        print("No comments returned")
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)

def getCommentReplies(serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, assetGUID, commentGUID):
    connectedAssetURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/framework-services/' + serviceURLMarker + '/connected-asset/users/' + userId 
    commentReplyQuery = connectedAssetURL + '/assets/' + assetGUID + '/comments/' + commentGUID + '/replies?elementStart=0&maxElements=50'
    response = issueGet(commentReplyQuery)
    responseObjects = response.json().get('list')
    if responseObjects:
        return responseObjects
    else:
        print("No comments returned")
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)


def getAPIOperations(serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, apiSchemaTypeGUID):
    connectedAssetURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/framework-services/' + serviceURLMarker + '/connected-asset/users/' + userId 
    requestURL = connectedAssetURL + '/assets/schemas/apis/' + apiSchemaTypeGUID + '/api-operations?elementStart=0&maxElements=50'
    response = issueGet(requestURL)
    responseObjects = response.json().get('list')
    if response.status_code == 200:
        relatedHTTPCode = response.json().get('relatedHTTPCode')
        if relatedHTTPCode == 200:
            return response.json().get('list')            
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    else:
        printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)

def getSchemaAttributesFromSchemaType(serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, schemaTypeGUID):
    ocfURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/framework-services/' + serviceURLMarker + '/connected-asset/users/' + userId
    getSchemaAttributesURL = ocfURL + '/assets/schemas/' + schemaTypeGUID + '/schema-attributes?elementStart=0&maxElements=100'
    response = issueGet(getSchemaAttributesURL)
    schemaAttributes = response.json().get('list')
    if schemaAttributes:
        return schemaAttributes
    else:
        print ("No Schema attributes retrieved")
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)

def printName(indent, name, guid):
    if not name:
        print(indent + "<null>" + " [" + guid + "]")
    else:
        print(indent + name + " [" + guid + "]")

def printType(indent, typeName, superTypeNames):
    if not superTypeNames:
        print(indent + "type: " + typeName)
    else:
        print(indent + "type: " + typeName + ' [%s]' % ', '.join(map(str, superTypeNames)))

def printStringProperty(indent, propertyName, propertyValue):
    if propertyValue:
        print(indent + propertyName + ": " + propertyValue)

def printMapProperties(indent, propertyName, properties):
    if properties:
        print(indent + propertyName + ":")
        propertyList = list(properties)
        propertyIndent = indent + "    "
        for x in range(len(propertyList)):
            propertyValue = properties.get(propertyList[x])
            print(propertyIndent + propertyList[x] + ": " + str(propertyValue))
    
def printMetadataCollection(indent, sourceServer, metadataCollectionType, metadataCollectionId, metadataCollectionName):
    print(indent + "origin:")
    print(indent + "    sourceServer:       " + sourceServer)
    if not metadataCollectionName:
        print(indent + "    metadataCollection: " + metadataCollectionId + " [" + metadataCollectionType + "]")
    else:
        print(indent + "    metadataCollection: " + metadataCollectionId + " [" + metadataCollectionType + ", " + metadataCollectionName + "]")

def printClassifications(indent, classifications, printOrigin):
    if classifications:
        print(indent + "classifications:")
        elementIndent = indent + "    "
        for classification in classifications:
            print(elementIndent + classification.get('classificationName'))
            propertyIndent = elementIndent + "    "
            if printOrigin:
                classificationElementOrigin = classification.get('origin')
                classificationSourceServer = classificationElementOrigin.get('sourceServer')
                classificationMetadataCollectionType = classificationElementOrigin.get('originCategory')
                classificationMetadataCollectionId = classificationElementOrigin.get('homeMetadataCollectionId')
                classificationMetadataCollectionName = classificationElementOrigin.get('homeMetadataCollectionName')
                printMetadataCollection(propertyIndent, classificationSourceServer, classificationMetadataCollectionType, classificationMetadataCollectionId, classificationMetadataCollectionName)
            properties = classification.get('classificationProperties')
            if properties:
                propertyList = list(properties)
                for x in range(len(propertyList)):
                    propertyValue = properties.get(propertyList[x])
                    print(propertyIndent + propertyList[x] + ": " + str(propertyValue))

def printMeanings(indent, meanings):
    if meanings:
        print(indent, "meanings:")
        propertyIndent = indent + "    "        
        for meaning in meanings:
            name = meaning.get('name')
            description = meaning.get('description')
            if description:
                print(propertyIndent + name + ": " + description)
            else:
                print(propertyIndent + name)

def printSearchKeywords(indent, keywords):
    if keywords:
        print(indent, "keywords:")
        propertyIndent = indent + "    "        
        for keyword in keywords:
            name = keyword.get('name')
            description = keyword.get('description')
            if description:
                print(propertyIndent + name + ": " + description)
            else:
                print(propertyIndent + name)

def printAsset(indent, asset, printOrigin):
    if asset:
        printName(indent, asset.get('name'), asset.get('guid'))
        propertyIndent = "    "
        printStringProperty(propertyIndent, "qualifiedName", asset.get('qualifiedName'))
        printStringProperty(propertyIndent, "versionIdentifier", asset.get('versionIdentifier'))
        printStringProperty(propertyIndent, "description", asset.get('description'))
        elementType = asset.get('type')
        printType(propertyIndent, elementType.get('typeName'), elementType.get('superTypeNames'))
        if printOrigin:
            elementOrigin = asset.get('origin')
            sourceServer = elementOrigin.get('sourceServer')
            metadataCollectionType = elementOrigin.get('originCategory')
            metadataCollectionId = elementOrigin.get('homeMetadataCollectionId')
            metadataCollectionName = elementOrigin.get('homeMetadataCollectionName')
            printMetadataCollection(propertyIndent, sourceServer, metadataCollectionType, metadataCollectionId, metadataCollectionName)
        elementVersions = asset.get('versions')
        printClassifications(propertyIndent, asset.get('classifications'), printOrigin)
        printMapProperties(propertyIndent, "additionalProperties", asset.get('additionalProperties'))
        printMapProperties(propertyIndent, "extendedProperties", asset.get('extendedProperties'))
        printMeanings(propertyIndent, asset.get('meanings'))
        printSearchKeywords(propertyIndent, asset.get('searchKeywords'))

def printSchemaType(indent, schemaType, printOrigin):
    if schemaType:
        print(indent + "schemaType")
        nameIndent = indent + "    "
        printName(nameIndent, schemaType.get('displayName'), schemaType.get('guid'))
        propertyIndent = nameIndent + "    "
        printStringProperty(propertyIndent, "qualifiedName", schemaType.get('qualifiedName'))
        printStringProperty(propertyIndent, "versionNumber", schemaType.get('versionNumber'))
        printStringProperty(propertyIndent, "description", schemaType.get('description'))
        printStringProperty(propertyIndent, "author", schemaType.get('author'))
        printStringProperty(propertyIndent, "usage", schemaType.get('usage'))
        printStringProperty(propertyIndent, "encodingStandard", schemaType.get('encodingStandard'))
        printStringProperty(propertyIndent, "namespace", schemaType.get('namespace'))
        elementType = schemaType.get('type')
        printType(propertyIndent, elementType.get('typeName'), elementType.get('superTypeNames'))
        if printOrigin:
            elementOrigin = schemaType.get('origin')
            sourceServer = elementOrigin.get('sourceServer')
            metadataCollectionType = elementOrigin.get('originCategory')
            metadataCollectionId = elementOrigin.get('homeMetadataCollectionId')
            metadataCollectionName = elementOrigin.get('homeMetadataCollectionName')
            printMetadataCollection(propertyIndent, sourceServer, metadataCollectionType, metadataCollectionId, metadataCollectionName)
        elementVersions = schemaType.get('versions')
        printClassifications(propertyIndent, schemaType.get('classifications'), printOrigin)
        printMapProperties(propertyIndent, "additionalProperties", schemaType.get('additionalProperties'))
        printMapProperties(propertyIndent, "extendedProperties", schemaType.get('extendedProperties'))
        printMeanings(propertyIndent, schemaType.get('meanings'))
        printSearchKeywords(propertyIndent, schemaType.get('searchKeywords'))
        
def printSchemaTypeDetail(indent, schemaType, serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, printOrigin):
    if schemaType:
        print(indent + "schemaType")
        nameIndent = indent + "    "
        printName(nameIndent, schemaType.get('displayName'), schemaType.get('guid'))
        propertyIndent = nameIndent + "    "
        printStringProperty(propertyIndent, "qualifiedName", schemaType.get('qualifiedName'))
        printStringProperty(propertyIndent, "versionNumber", schemaType.get('versionNumber'))
        printStringProperty(propertyIndent, "description", schemaType.get('description'))
        printStringProperty(propertyIndent, "author", schemaType.get('author'))
        printStringProperty(propertyIndent, "usage", schemaType.get('usage'))
        printStringProperty(propertyIndent, "encodingStandard", schemaType.get('encodingStandard'))
        printStringProperty(propertyIndent, "namespace", schemaType.get('namespace'))
        elementType = schemaType.get('type')
        printType(propertyIndent, elementType.get('typeName'), elementType.get('superTypeNames'))
        if printOrigin:
            elementOrigin = schemaType.get('origin')
            sourceServer = elementOrigin.get('sourceServer')
            metadataCollectionType = elementOrigin.get('originCategory')
            metadataCollectionId = elementOrigin.get('homeMetadataCollectionId')
            metadataCollectionName = elementOrigin.get('homeMetadataCollectionName')
            printMetadataCollection(propertyIndent, sourceServer, metadataCollectionType, metadataCollectionId, metadataCollectionName)
        elementVersions = schemaType.get('versions')
        printClassifications(propertyIndent, schemaType.get('classifications'), printOrigin)
        printMapProperties(propertyIndent, "additionalProperties", schemaType.get('additionalProperties'))
        printMapProperties(propertyIndent, "extendedProperties", schemaType.get('extendedProperties'))
        printMeanings(propertyIndent, schemaType.get('meanings'))
        printSearchKeywords(propertyIndent, schemaType.get('searchKeywords'))
        schemaTypeClass = schemaType.get('class')
        if schemaTypeClass:
            if schemaTypeClass == "ComplexSchemaType":
                printSchemaAttributes(propertyIndent, serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, schemaType.get('guid'), printOrigin)
            else:
                if schemaTypeClass == "SchemaTypeChoice":
                    schemaOptions = schemaType.get('schemaOptions')
                    if schemaOptions:
                        print(propertyIndent + "Schema Options:")
                        for x in range(len(schemaOptions)):
                            schemaOptionIndent = propertyIndent + "    "
                            printSchemaTypeDetail(schemaOptionIndent, schemaOptions[x], serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, printOrigin)
                else:
                    if schemaTypeClass == "APISchemaType":
                        apiOperations = getAPIOperations(serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, schemaType.get('guid'))
                        if apiOperations:
                            print(propertyIndent + "API Operations:")
                            apiOperationIndent = propertyIndent + "    "
                            for x in range(len(apiOperations)):
                                apiOperation = apiOperations[x].get('apioperation')
                                if apiOperation:
                                    printSchemaTypeDetail(apiOperationIndent, apiOperation, serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, printOrigin)
                    else:
                        if schemaTypeClass == "APIOperation":
                            printStringProperty(propertyIndent, "API Operation", schemaType.get('command'))
                            parameterListIndent = propertyIndent + "    "
                            apiHeader = schemaType.get('headerSchemaType')
                            if apiHeader:
                                print(propertyIndent + "header:")
                                printSchemaTypeDetail(parameterListIndent, apiHeader, serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, printOrigin)
                            apiRequest = schemaType.get('requestSchemaType')
                            if apiRequest:
                                print(propertyIndent + "request:")
                                printSchemaTypeDetail(parameterListIndent, apiRequest, serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, printOrigin)
                            apiResponse = schemaType.get('responseSchemaType')
                            if apiResponse:
                                print(propertyIndent + "response:")
                                printSchemaTypeDetail(parameterListIndent, apiResponse, serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, printOrigin)



def printSchemaAttributes(indent, serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, schemaTypeGUID, printOrigin):
    schemaAttributes = getSchemaAttributesFromSchemaType(serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, schemaTypeGUID)
    if schemaAttributes:
        print(indent + "schemaAttributes:")
        elementIndent = indent + "    "
        for x in range(len(schemaAttributes)):
            printSchemaAttribute(elementIndent, schemaAttributes[x], printOrigin)
            schemaType = schemaAttributes[x].get('attributeType')
            if schemaType:
                schemaTypeIndent = elementIndent + "    "
                printNestedSchemaAttribute(schemaTypeIndent, schemaType, serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, printOrigin)

def printSchemaAttribute(indent, schemaAttribute, printOrigin):
    if schemaAttribute:
        nameIndent = indent
        printName(nameIndent, schemaAttribute.get('displayName'), schemaAttribute.get('guid'))
        propertyIndent = nameIndent + "    "
        elementType = schemaAttribute.get('type')
        printType(propertyIndent, elementType.get('typeName'), elementType.get('superTypeNames'))
        if printOrigin:
            elementOrigin = schemaAttribute.get('origin')
            sourceServer = elementOrigin.get('sourceServer')
            metadataCollectionType = elementOrigin.get('originCategory')
            metadataCollectionId = elementOrigin.get('homeMetadataCollectionId')
            metadataCollectionName = elementOrigin.get('homeMetadataCollectionName')
            printMetadataCollection(propertyIndent, sourceServer, metadataCollectionType, metadataCollectionId, metadataCollectionName)
        elementVersions = schemaAttribute.get('versions')
        printClassifications(propertyIndent, schemaAttribute.get('classifications'), printOrigin)
        printMapProperties(propertyIndent, "additionalProperties", schemaAttribute.get('additionalProperties'))
        printMapProperties(propertyIndent, "extendedProperties", schemaAttribute.get('extendedProperties'))
        printMeanings(propertyIndent, schemaAttribute.get('meanings'))
        printSearchKeywords(propertyIndent, schemaAttribute.get('searchKeywords'))

    
def printNestedSchemaAttribute(indent, schemaType, serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, printOrigin):
    schemaTypeClass = schemaType.get('class')
    if schemaTypeClass:
         if schemaTypeClass == "ComplexSchemaType":
            printSchemaAttributes(indent, serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, schemaType.get('guid'), printOrigin)


def printRelatedAssets(indent, serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, assetGUID):
    relatedAssets = getRelatedAssets(serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, assetGUID)
    if relatedAssets:      
        print(indent + "relatedAssets:")
        elementIndent = indent + "    "
        for x in range(len(relatedAssets)):
            printRelatedAsset(elementIndent, relatedAssets[x])

def printRelatedAsset(indent, relatedAsset):
    printName(indent, relatedAsset.get('name'), relatedAsset.get('guid'))
    propertyIndent = indent + "    "
    printStringProperty(propertyIndent, "relationshipName", relatedAsset.get('relationshipName'))
    printStringProperty(propertyIndent, "attributeName", relatedAsset.get('attributeName'))
    printStringProperty(propertyIndent, "qualifiedName", relatedAsset.get('qualifiedName'))
    printStringProperty(propertyIndent, "versionNumber", relatedAsset.get('versionNumber'))
    printStringProperty(propertyIndent, "description", relatedAsset.get('description'))
    elementType = relatedAsset.get('type')
    printType(propertyIndent, elementType.get('typeName'), elementType.get('superTypeNames'))

def printComments(indent, serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, assetGUID, printOrigin):
    comments = getComments(serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, assetGUID)
    if comments:      
        print(indent + "comments:")
        elementIndent = indent + "    "
        replyIndent = elementIndent + "    "
        for x in range(len(comments)):
            comment = comments[x].get('comment')
            commentGUID = comment.get('guid')
            replyCount = printCommentResponse(elementIndent, comments[x], printOrigin)
            if replyCount:
                if replyCount > 0:
                    printCommentReplies(replyIndent, serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, assetGUID, commentGUID, printOrigin)

def printCommentReplies(indent, serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, assetGUID, commentGUID, printOrigin):
    commentReplies = getCommentReplies(serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, assetGUID, commentGUID)
    if commentReplies:
        print(indent + "replies:")
        elementIndent = indent + "    "
        replyIndent = elementIndent + "    "
        for x in range(len(commentReplies)):
            comment = commentReplies[x].get('comment')
            replyGUID = comment.get('guid')
            replyCount = printCommentResponse(replyIndent, commentReplies[x], printOrigin)
            if replyCount:
                if replyCount > 0:
                    printCommentReplies(replyIndent, serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, assetGUID, replyGUID, printOrigin)

def printCommentResponse(indent, commentObject, printOrigin):
    if commentObject:
        comment   = commentObject.get('comment')
        replyCount = commentObject.get('replyCount')
        if comment:
            commentIndent = indent + "    "
            print(indent + "comment")
            commentType = comment.get('commentType')
            commentText = comment.get('commentText') 
            commentUser = comment.get('user') 
            isPublic    = comment.get('isPublic') 
            if commentType:
                print(commentIndent + "comment type: " + commentType)
            if commentText:
                print(commentIndent + "comment text: " + commentText)
            if commentUser:
                print(commentIndent + "created by:   " + commentUser)
            if isPublic:
                print(commentIndent + "public:       " + str(isPublic))
            if printOrigin:
                elementOrigin = comment.get('origin')
                sourceServer = elementOrigin.get('sourceServer')
                metadataCollectionType = elementOrigin.get('originCategory')
                metadataCollectionId = elementOrigin.get('homeMetadataCollectionId')
                metadataCollectionName = elementOrigin.get('homeMetadataCollectionName')
                printMetadataCollection(commentIndent, sourceServer, metadataCollectionType, metadataCollectionId, metadataCollectionName)
            return replyCount  
        else:
            print ("Returns:")
            prettyResponse = json.dumps(commentObject, indent=4)
            print (prettyResponse)
            print (" ")      


def printAssetUniverse(serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, assetGUID):
    printSelectiveAssetUniverse(serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, assetGUID, True, True)

    
def printSelectiveAssetUniverse(serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, assetGUID, printOrigin, printSchema):
    assetResponse = getAssetUniverse(serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, assetGUID)
    if assetResponse:
        printAsset("", assetResponse.get('asset'), printOrigin)
        propertyIndent = "    "
        if printSchema:
            printSchemaTypeDetail(propertyIndent, assetResponse.get('schemaType'), serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, printOrigin)
        certificationCount = assetResponse.get('certificationCount')
        commentCount = assetResponse.get('commentCount')
        if commentCount > 0:
            printComments(propertyIndent, serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, assetGUID, printOrigin)
        connectionCount = assetResponse.get('connectionCount')
        externalIdentifierCount = assetResponse.get('externalIdentifierCount')
        externalReferencesCount = assetResponse.get('externalReferencesCount')
        informalTagCount = assetResponse.get('informalTagCount')
        licenseCount = assetResponse.get('licenseCount')
        likeCount = assetResponse.get('likeCount')
        keywordCount = assetResponse.get('keywordCount')
        knownLocationsCount = assetResponse.get('knownLocationsCount')
        noteLogsCount = assetResponse.get('noteLogsCount')
        ratingsCount = assetResponse.get('ratingsCount')
        relatedAssetCount = assetResponse.get('relatedAssetCount')
        if relatedAssetCount > 0:
            printRelatedAssets(propertyIndent, serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, assetGUID)
        relatedMediaReferenceCount = assetResponse.get('relatedMediaReferenceCount')
       
            
def printCommentList(responseObjects):
    print(" ")
    if len(responseObjects) == 0:
        print ("No comments found")
    else:
        if len(responseObjects) == 1:
            print ("1 comment found:")
        else:
            print (str(len(responseObjects)) + " comments found:")
        for x in range(len(responseObjects)):
            printComment(responseObjects[x]) 


def printAssetComments(serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, requestType):
    connectedAssetURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/framework-services/' + serviceURLMarker + '/connected-asset/users/' + userId 
    commentQuery = connectedAssetURL + '/assets/' + requestType + '/comments?elementStart=0&maxElements=50'
    response = issueGet(commentQuery)
    responseObjects = response.json().get('list')
    if (responseObjects):
        printCommentList(responseObjects)
    else:
        print("No comments returned")
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)
    
def printAssetCommentReplies(serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, requestType, commentGUID):
    connectedAssetURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/framework-services/' + serviceURLMarker + '/connected-asset/users/' + userId 
    commentReplyQuery = connectedAssetURL + '/assets/' + requestType + '/comments/' + commentGUID + '/replies?elementStart=0&maxElements=50'
    response = issueGet(commentReplyQuery)
    responseObjects = response.json().get('list')
    if (responseObjects):
        printCommentList(responseObjects)
    else:
        print("No comments returned")
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)

def printComment(commentObject):
    if commentObject:
        comment   = commentObject.get('comment')
        relyCount = commentObject.get('replyCount')
        if comment:
            commentType = comment.get('commentType')
            commentText = comment.get('commentText') 
            commentUser = comment.get('user') 
            isPublic    = comment.get('isPublic') 
            if commentType:
                print("  comment type: " + commentType)
            if commentText:
                print("  comment text: " + commentText)
            if commentUser:
                print("  created by:   " + commentUser)
            if isPublic:
                print("  public:       " + str(isPublic))



In [ ]:
   
#
# Working with assets - this set of functions are for authoring assets.
# Using the Asset Owner OMAS interface to create and query assets.  Notice that the interface returns all of the asset contents.
# 

def assetOwnerCreateAsset(assetTypeURL, serverName, serverPlatformName, serverPlatformURL, userId, displayName, description, fullPath):
    assetOwnerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-owner/users/' + userId
    createAssetURL = assetOwnerURL + assetTypeURL       
    createAssetBody = {
        "class" : "NewCSVFileAssetRequestBody",
        "displayName" : displayName,
        "description" : description,
        "fullPath" : fullPath
    }
    response=issuePost(createAssetURL, createAssetBody)
    guids = response.json().get('guids')
    if guids:
        return guids
    else:
        print ("No assets created")
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)
    
def assetOwnerCreateAsset(assetTypeURL, serverName, serverPlatformName, serverPlatformURL, userId, displayName, description, fullPath):
    assetOwnerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-owner/users/' + userId
    createAssetURL = assetOwnerURL + assetTypeURL
    jsonHeader = {'content-type':'application/json'}
    createAssetBody = {
        "class" : "NewCSVFileAssetRequestBody",
        "displayName" : displayName,
        "description" : description,
        "fullPath" : fullPath
    }
    response=issuePost(createAssetURL, createAssetBody)
    guids = response.json().get('guids')
    if guids:
        return guids
    else:
        print ("No assets created")
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)
    
def assetOwnerCreateCSVAsset(serverName, serverPlatformName, serverPlatformURL, userId, displayName, description, fullPath):
    return assetOwnerCreateAsset('/assets/data-files/csv', serverName, serverPlatformName, serverPlatformURL, userId, displayName, description, fullPath)

def assetOwnerCreateAvroAsset(serverName, serverPlatformName, serverPlatformURL, userId, displayName, description, fullPath):
    return assetOwnerCreateAsset('/assets/data-files/avro', serverName, serverPlatformName, serverPlatformURL, userId, displayName, description, fullPath)

def assetOwnerCreateCSVAssetWithColumnHeaders(serverName, serverPlatformName, serverPlatformURL, userId, displayName, description, fullPath, columnHeaders):
    assetOwnerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-owner/users/' + userId
    createAssetURL = assetOwnerURL + '/assets/data-files/csv'
    createAssetBody = {
        "class" : "NewCSVFileAssetRequestBody",
        "displayName" : displayName,
        "description" : description,
        "fullPath" : fullPath,
        "columnHeaders" : columnHeaders
    }
    response=issuePost(createAssetURL, createAssetBody)
    guids = response.json().get('guids')
    if guids:
        return guids
    else:
        print ("No CSV assets created")
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)

def assetOwnerGetSchemaAttributesFromSchemaType(serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, schemaTypeGUID):
    return getSchemaAttributesFromSchemaType(serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, schemaTypeGUID)
        
# Delete the asset with the supplied guid.     
def assetOwnerDelete(serverName, serverPlatformName, serverPlatformURL, userId, assetGUID):
    assetOwnerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-owner/users/' + userId
    deleteAssetURL = assetOwnerURL + '/assets/"+ assetGUID + "/delete'
    response=issuePost(deleteAssetURL, {})

    if relatedHTTPCode == 200:
        print ("deleted Asset")
        return []
    else:    
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)
        
def assetOwnerSearchForAssets(serverName, serverPlatformName, serverPlatformURL, userId, searchString):
    assetOwnerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-owner/users/' + userId
    getAssetsURL = assetOwnerURL + '/assets/by-search-string?startFrom=0&pageSize=50'
    searchBody = {
        "class" : "SearchStringRequestBody",
        "searchString" : searchString
    }
    response = issuePost(getAssetsURL, searchBody)
    if response:
        assets = response.json().get('assets')
        if assets:
            return assets
        else:
            print ("No assets found")
            processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)
    else:
        print ("Search Request Failed")        

def assetOwnerPrintAssets(serverName, serverPlatformName, serverPlatformURL, userId, searchString):
    assets = assetOwnerSearchForAssets(serverName, serverPlatformName, serverPlatformURL, userId, searchString)
    if assets:
        if len(assets) == 1:
            print ("1 asset found:")
        else:
            print (str(len(assets)) + " assets found:")
        printAssetListDetail(assets)
        
def assetOwnerFindAssetQualifiedName(serverName, serverPlatformName, serverPlatformURL, userId, searchString):
    qualifiedName = None
    assets = None
    assets = assetOwnerSearchForAssets(serverName, serverPlatformName, serverPlatformURL, userId, searchString)
    if assets == None:
        time.sleep(1)
        assets = assetOwnerSearchForAssets(serverName, serverPlatformName, serverPlatformURL, userId, searchString)
    if assets:
        if len(assets) == 1:
            assetProperties = assets[0].get('assetProperties')
            qualifiedName = assetProperties.get('qualifiedName')
        else:
            print (str(len(assets)) + " assets found:")
    return qualifiedName

def assetOwnerFindAssetPathName(serverName, serverPlatformName, serverPlatformURL, userId, searchString):
    pathName = None
    assets = None
    assets = assetOwnerSearchForAssets(serverName, serverPlatformName, serverPlatformURL, userId, searchString)
    if assets == None:
        time.sleep(1)
        assets = assetOwnerSearchForAssets(serverName, serverPlatformName, serverPlatformURL, userId, searchString)
    if assets:
        if len(assets) == 1:
            assetProperties = assets[0].get('assetProperties')
            extendedProperties = assetProperties.get('extendedProperties')
            if extendedProperties:
                pathName = extendedProperties.get('pathName')
        else:
            print (str(len(assets)) + " assets found:")
    return pathName

def assetOwnerDeleteAssets(serverName, serverPlatformName, serverPlatformURL, userId, searchString):
    assets = assetOwnerSearchForAssets(serverName, serverPlatformName, serverPlatformURL, userId, searchString)
    if assets:
        if len(assets) == 1:
            print ("1 asset to delete")
        else:
            print (str(len(assets)) + " assets to delete:")
        for asset in assets:
            elementHeader = asset.get('elementHeader')
            assetGUID = elementHeader.get('guid')
            assetProperties = asset.get('assetProperties')
            assetQualifiedName = assetProperties.get('qualifiedName')
            print("Deleting asset " + assetQualifiedName)
            assetOwnerCreateAssetDelete(serverName, serverPlatformName, serverPlatformURL, userId, assetGUID)

def addOwner(serverName, serverPlatformName, serverPlatformURL, userId, assetName, assetGUID, assetOwner, ownerType):
    governanceURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-owner/users/' + userId 
    print ("Setting owner on " + assetName + " to " + assetOwner + " ...")
    body = {
        "class" : "OwnerRequestBody",
        "ownerType" : ownerType,
        "ownerId" : assetOwner
    }
    addOwnerURL = governanceURL + "/assets/" + assetGUID + "/owner"
    response=issuePost(addOwnerURL, body)
    if response.status_code != 200:
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)
        
def addOrigin(serverName, serverPlatformName, serverPlatformURL, userId, assetName, assetGUID, contact, originatingDept, originatingOrganization):
    governanceURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-owner/users/' + userId 
    print ("Setting origin on " + assetName + " ...")
    body = {
        "class" : "OriginRequestBody",
        "otherOriginValues" : {
            "originatingOrganization" : originatingOrganization,
            "originatingDept" : originatingDept,
            "contact" : contact
        }
    }
    addOriginURL = governanceURL + "/assets/" + assetGUID + "/origin"
    response=issuePost(addOriginURL, body)
    if response.status_code != 200:
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)
        
def addZones(serverName, serverPlatformName, serverPlatformURL, userId, assetName, assetGUID, zones):
    print ("Setting governance zones on " + assetName + " ...")
    governanceURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-owner/users/' + userId 
    
    addZonesURL = governanceURL + "/assets/" + assetGUID + "/governance-zones"
    response=issuePost(addZonesURL, zones)
    if response.status_code != 200:
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)

                      
# Create a semantic assignment between an Asset's attachment - like a Schema Type - and Glossary Term 
def createSemanticAssignment(serverName, serverPlatformName, serverPlatformURL, userId, assetGUID, schemaGUID, glossaryTermGUID):                
    assetOwnerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-owner/users/' + userId
    semanticAssignmentURL =  assetOwnerURL + '/assets/' + assetGUID + '/attachments/' + schemaGUID + '/meanings/' + glossaryTermGUID
    response=issuePost(semanticAssignmentURL, {})
    if response.status_code == 200:
        print("Semantic assignment relationship created") 
    else:
        print ("No semantic assignment Relationship created")
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)


In [ ]:
#
# Asset Consumer OMAS
#

# Retrieve the OutTopic connection

def getOutTopicConnection(serverName, serverPlatformName, serverPlatformURL, userId, callerId):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    requestURL = assetConsumerURL + '/topics/out-topic-connection/' + callerId
    response=issueGet(requestURL)
    connection = response.json().get('connection')
    if connection:
        return connection
    else:
        print ("No connection found")
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)

# Finding Assets

def getAssetForConnectionName(serverName, serverPlatformName, serverPlatformURL, userId, connectionName):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    requestURL = assetConsumerURL + '/assets/by-connection-name/' + connectionName
    response=issueGet(requestURL)
    guid = response.json().get('guid')
    if guid:
        return guid
    else:
        print ("No asset found")
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)

def getAssetsByMeaning(serverName, serverPlatformName, serverPlatformURL, userId, termGUID):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    requestURL = assetConsumerURL + '/assets/by-meaning/' + termGUID + '?startFrom=0&pageSize=50'
    response=issueGet(requestURL)
    guids = response.json().get('guids')
    if guids:
        return guids
    else:
        print ("No assets found")
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)

def getAssetsByTag(serverName, serverPlatformName, serverPlatformURL, userId, tagGUID):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    requestURL = assetConsumerURL + '/assets/by-tag/' + tagGUID + '?startFrom=0&pageSize=50'
    response=issueGet(requestURL)
    guids = response.json().get('guids')
    if guid:
        return guids
    else:
        print ("No assets found")
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)

def getAssetsByName(serverName, serverPlatformName, serverPlatformURL, userId, name):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    requestURL = assetConsumerURL + '/assets/by-name?startFrom=0&pageSize=50'
    searchBody = {
        "class" : "NameRequestBody",
        "name" : name
    }
    response = issuePost(requestURL, searchBody)
    guids = response.json().get('guids')
    if guid:
        return guids
    else:
        print ("No assets found")
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)

def searchForAssets(serverName, serverPlatformName, serverPlatformURL, userId, searchString):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    requestURL = assetConsumerURL + '/assets/by-search-string?startFrom=0&pageSize=50'
    searchBody = {
        "class" : "SearchStringRequestBody",
        "searchString" : searchString
    }
    response = issuePost(requestURL, searchBody)
    guids = response.json().get('guids')
    if guids:
        return guids
    else:
        print ("No assets found")
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)

def assetConsumerSearchForAssets(serverName, serverPlatformName, serverPlatformURL, userId, searchString):
    return searchForAssets(serverName, serverPlatformName, serverPlatformURL, userId, searchString)

def assetConsumerPrintAssets(serverName, serverPlatformName, serverPlatformURL, userId, searchString): 
    guids = assetConsumerSearchForAssets(serverName, serverPlatformName, serverPlatformURL, userId, searchString)
    if guids:
        printGUIDList(guids)  


# Working with comments

def addCommentToAsset(serverName, serverPlatformName, serverPlatformURL, userId, assetGUID, commentText, commentType, isPublic):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    requestURL = assetConsumerURL + '/assets/' + assetGUID + '/comments'
    commentBody={
        "class" : "CommentRequestBody",
        "commentType" : commentType,
        "commentText" : commentText,
        "isPublic" : isPublic
    }
    response = issuePost(requestURL,commentBody)
    commentGUID = response.json().get('guid')
    if commentGUID:
        return commentGUID
    else:
        print ("No comment added")
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)


def updateComment(serverName, serverPlatformName, serverPlatformURL, userId, assetGUID, commentGUID, commentText, commentType, isPublic):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    requestURL = assetConsumerURL + '/assets/' + assetGUID + '/comments/' + commentGUID + '/update'
    commentBody={
        "class" : "CommentRequestBody",
        "commentType" : commentType,
        "commentText" : commentText,
        "isPublic" : isPublic
    }
    response = issuePost(requestURL,commentBody)
    processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)


def addReplyToAssetComment(serverName, serverPlatformName, serverPlatformURL, userId, assetGUID, commentGUID, commentText, commentType, isPublic):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    requestURL = assetConsumerURL + '/assets/' + assetGUID + '/comments/' + commentGUID + '/replies'
    requestBody={
        "class" : "CommentRequestBody",
        "commentType" : commentType,
        "commentText" : commentText,
        "isPublic" : isPublic
    }
    response = issuePost(requestURL, requestBody)
    commentGUID = response.json().get('guid')
    if commentGUID:
        return commentGUID
    else:
        print ("No comment added")
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)


def assetConsumerAddReplyToAssetComment(serverName, serverPlatformName, serverPlatformURL, userId, assetGUID, commentGUID, commentText, commentType, isPublic):
    return addReplyToAssetComment(serverName, serverPlatformName, serverPlatformURL, userId, assetGUID, commentGUID, commentText, commentType, isPublic)


def removeCommentFromAsset(serverName, serverPlatformName, serverPlatformURL, userId, assetGUID, commentGUID):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    requestURL = assetConsumerURL + '/assets/' + assetGUID + '/comments/' + commentGUID + '/delete'
    requestBody={
        "class" : "NullRequestBody",
    }
    response = issuePost(requestURL, requestBody)
    processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)


# Working with glossary terms (meanings)   

def getMeaningsByName(serverName, serverPlatformName, serverPlatformURL, userId, name):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    requestURL = assetConsumerURL + '/meanings/by-name?startFrom=0&pageSize=50'
    searchBody = {
        "class" : "NameRequestBody",
        "name" : name
    }
    response = issuePost(requestURL, searchBody)
    meanings = response.json().get('meanings')
    if meanings:
        return meanings
    else:
        print ("No meanings found")
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)
        

def findMeanings(serverName, serverPlatformName, serverPlatformURL, userId, searchString):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    requestURL = assetConsumerURL + '/meanings/by-search-string?startFrom=0&pageSize=50'
    searchBody = {
        "class" : "SearchStringRequestBody",
        "searchString" : searchString
    }
    response = issuePost(requestURL, searchBody)
    meanings = response.json().get('meanings')
    if meanings:
        return meanings
    else:
        print ("No meanings found")
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)
        

def getMeaning(serverName, serverPlatformName, serverPlatformURL, userId, termGUID):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    requestURL = assetConsumerURL + '/meanings/' + termGUID
    response=issueGet(requestURL)
    meaning = response.json().get('meaning')
    if meaning:
        return meaning
    else:
        print ("No meaning found")
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)

# Working with Informal Tags

def addTagToAsset(serverName, serverPlatformName, serverPlatformURL, userId, assetGUID, tagGUID, isPublic):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    requestURL = assetConsumerURL + '/assets/' + assetGUID + '/tags/' + tagGUID
    requestBody={
        "class" : "FeedbackRequestBody",
        "isPublic" : isPublic
    }
    response = issuePost(requestURL,requestBody)
    processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)

    
def addTagToElement(serverName, serverPlatformName, serverPlatformURL, userId, elementGUID, tagGUID, isPublic):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    requestURL = assetConsumerURL + '/assets/elements/' + elementGUID + '/tags/' + tagGUID
    requestBody={
        "class" : "FeedbackRequestBody",
        "isPublic" : isPublic
    }
    response = issuePost(requestURL,requestBody)
    processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)


def removeTagFromAsset(serverName, serverPlatformName, serverPlatformURL, userId, assetGUID, tagGUID):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    requestURL = assetConsumerURL + '/assets/' + assetGUID + '/tags/' + tagGUID + '/delete'
    requestBody={
        "class" : "NullRequestBody",
    }
    response = issuePost(requestURL, requestBody)
    processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)


def removeTagFromElement(serverName, serverPlatformName, serverPlatformURL, userId, elementGUID, tagGUID):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    requestURL = assetConsumerURL + '/assets/elements/' + elementGUID + '/tags/' + tagGUID + '/delete'
    requestBody={
        "class" : "NullRequestBody",
    }
    response = issuePost(requestURL, requestBody)
    processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)


def createTag(serverName, serverPlatformName, serverPlatformURL, userId, isPrivateTag, name, description):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    requestURL = assetConsumerURL + '/tags'
    requestBody={
        "class" : "TagRequestBody",
        "isPrivateTag" : isPrivateTag,
        "name" : name,
        "description" : description,
        "user" : userId
    }
    response = issuePost(requestURL,requestBody)
    tagGUID = response.json().get('guid')
    if tagGUID:
        return tagGUID
    else:
        print ("No tag added")
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)


def updateTagDescription(serverName, serverPlatformName, serverPlatformURL, userId, tagGUID, description):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    requestURL = assetConsumerURL + '/tags/' + tagGUID + '/update'
    requestBody={
        "class" : "TagUpdateRequestBody",
        "description" : description,
    }
    response = issuePost(requestURL,requestBody)
    processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)


def deleteTag(serverName, serverPlatformName, serverPlatformURL, userId, tagGUID):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    requestURL = assetConsumerURL + '/tags/' + tagGUID + '/delete'
    requestBody={
        "class" : "NullRequestBody"
    }
    response = issuePost(requestURL,requestBody)
    processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)


def getTagsByName(serverName, serverPlatformName, serverPlatformURL, userId, name):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    requestURL = assetConsumerURL + '/tags/by-name?startFrom=0&pageSize=50'
    searchBody = {
        "class" : "NameRequestBody",
        "name" : name
    }
    response = issuePost(requestURL, searchBody)
    tags = response.json().get('tags')
    if tags:
        return tags
    else:
        print ("No tags found")
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)


def getMyTagsByName(serverName, serverPlatformName, serverPlatformURL, userId, name):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    requestURL = assetConsumerURL + '/tags/private/by-name?startFrom=0&pageSize=50'
    searchBody = {
        "class" : "NameRequestBody",
        "name" : name
    }
    response = issuePost(requestURL, searchBody)
    tags = response.json().get('tags')
    if tags:
        return tags
    else:
        print ("No tags found")
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)
        

def findTags(serverName, serverPlatformName, serverPlatformURL, userId, searchString):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    requestURL = assetConsumerURL + '/tags/by-search-string?startFrom=0&pageSize=50'
    searchBody = {
        "class" : "SearchStringRequestBody",
        "searchString" : searchString
    }
    response = issuePost(requestURL, searchBody)
    tags = response.json().get('tags')
    if tags:
        return tags
    else:
        print ("No tags found")
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)


def findMyTags(serverName, serverPlatformName, serverPlatformURL, userId, searchString):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    requestURL = assetConsumerURL + '/tags/private/by-search-string?startFrom=0&pageSize=50'
    searchBody = {
        "class" : "SearchStringRequestBody",
        "searchString" : searchString
    }
    response = issuePost(requestURL, searchBody)
    tags = response.json().get('tags')
    if tags:
        return tags
    else:
        print ("No tags found")
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)


def getTag(serverName, serverPlatformName, serverPlatformURL, userId, tagGUID):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    requestURL = assetConsumerURL + '/tags/' + tagGUID
    response=issueGet(requestURL)
    tag = response.json().get('tag')
    if tag:
        return tag
    else:
        print ("No tag found")
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)

# Working with Ratings/Reviews

def addRatingToAsset(serverName, serverPlatformName, serverPlatformURL, userId, assetGUID, starRating, review, isPublic):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    requestURL = assetConsumerURL + '/assets/' + assetGUID + '/ratings'
    requestBody={
        "class" : "RatingRequestBody",
        "starRating" : starRating,
        "review" : review,
        "user" : userId,
        "isPublic" : isPublic
    }
    response = issuePost(requestURL,requestBody)
    processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)


def removeRatingFromAsset(serverName, serverPlatformName, serverPlatformURL, userId, assetGUID):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    requestURL = assetConsumerURL + '/assets/' + assetGUID + '/ratings/delete'
    requestBody={
        "class" : "NullRequestBody",
    }
    response = issuePost(requestURL, requestBody)
    processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)

    
# Working with Likes

def addLikeToAsset(serverName, serverPlatformName, serverPlatformURL, userId, assetGUID, isPublic):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    requestURL = assetConsumerURL + '/assets/' + assetGUID + '/likes'
    requestBody={
        "class" : "FeedbackRequestBody",
        "isPublic" : isPublic
    }
    response = issuePost(requestURL,requestBody)
    processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)


def removeLikeFromAsset(serverName, serverPlatformName, serverPlatformURL, userId, assetGUID):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    requestURL = assetConsumerURL + '/assets/' + assetGUID + '/likes/delete'
    requestBody={
        "class" : "NullRequestBody",
    }
    response = issuePost(requestURL, requestBody)
    processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)



# Working with the Audit log

def addLogMessageToAsset(serverName, serverPlatformName, serverPlatformURL, userId, assetGUID, connectorInstanceId, connectionName, connectorType, contextId, message):
    assetConsumerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-consumer/users/' + userId 
    requestURL = assetConsumerURL + '/assets/' + requestType + '/comments'
    requestBody={
        "class" : "LogRecordRequestBody",
        "connectorInstanceId" : connectorInstanceId,
        "connectionName" : connectionName,
        "connectorType" : connectorType,
        "contextId" : contextId,
        "message" : message
    }
    response = issuePost(requestURL,requestBody)
    processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)


#
# These methods call the OCG Metadata Management Interface on behalf of Asset Consumer OMAS
#
def assetConsumerGetAssetUniverse(serverName, serverPlatformName, serverPlatformURL, userId, requestType):
    return getAssetUniverse(serverName, serverPlatformName, serverPlatformURL, "asset-consumer", userId, requestType)
        
def assetConsumerPrintAssetUniverse(serverName, serverPlatformName, serverPlatformURL, userId, requestType):
    printAssetUniverse(serverName, serverPlatformName, serverPlatformURL, "asset-consumer", userId, requestType)

def assetConsumerPrintAssetCommentReplies(serverName, serverPlatformName, serverPlatformURL, userId, requestType, commentGUID):
    printAssetCommentReplies(serverName, serverPlatformName, serverPlatformURL, "asset-consumer", userId, requestType, commentGUID)

def assetConsumerPrintRelatedAssets(serverName, serverPlatformName, serverPlatformURL, userId, requestType):
    printRelatedAssets(serverName, serverPlatformName, serverPlatformURL, "asset-consumer", userId, requestType)

def assetConsumerPrintAssetComments(serverName, serverPlatformName, serverPlatformURL, userId, requestType):
    printAssetComments(serverName, serverPlatformName, serverPlatformURL, "asset-consumer", userId, requestType)


In [ ]:
#
# Asset Manager OMAS
# This function assumes the assets exist, and it creates the lineage between the source asset schema type, to the port to the process to the port to 
# the target asset schematype
#
def createDirectedProcessBetweenAssets(serverName, serverPlatformName, serverPlatformURL, userId, sourceSchemaTypeGuid, targetSchemaTypeGuid, processName, processDescription, processFormula):

    assetManagerURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/asset-manager/users/' + userId
    createProcessURL = assetManagerURL + '/processes?assetManagerIsHome=false&initialStatus=ACTIVE'
   
    elementProperties = {
        'class':'ProcessProperties',
        'qualifiedName' : processName,
        'technicalName':processName,
        'technicalDescription':processDescription,
        'formula': processFormula
    } 
    metadataCorrelationProperties = {
        'class':'MetadataCorrelationProperties'
    }
    # create the process
    processRequestBody= {
        'class':'ProcessRequestBody',
        'elementProperties': elementProperties,
        'metadataCorrelationProperties': metadataCorrelationProperties
        
    }    
    response=issuePost(createProcessURL, processRequestBody)
    processGuid = response.json().get('guid')
    if processGuid:
        # create in port
        createPortURL = assetManagerURL + '/processes/' + processGuid + '/ports?assetManagerIsHome=false'
        metadataCorrelationProperties= {
            'class':'MetadataCorrelationProperties'
        }
        inPortProperties = {
            'class':'PortProperties',
            'qualifiedName': processGuid + "-in-port",
            'portType': 'INPUT_PORT',
            'displayName': 'in port'
        }
        inPortRequestBody = {
            'class':'PortRequestBody',
            'MetadataCorrelationProperties': metadataCorrelationProperties,
            'elementProperties': inPortProperties
        } 
        
        response=issuePost(createPortURL, inPortRequestBody)
        inPortGuid = response.json().get('guid')
        if inPortGuid is None:
            print ("No in port created")
            if response.status_code != 200:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
                return [] 
            else:
                relatedHTTPCode = response.json().get('relatedHTTPCode')
                if relatedHTTPCode != 200:
                    printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
                return []     
        
        # create in portschema
        createInPortSchemaURL = assetManagerURL + '/ports/' + inPortGuid + '/schema-type/' + sourceSchemaTypeGuid +'?assetManagerIsHome=false'
        assetManagerIdentifiersRequestBody = {
            'class':'AssetManagerIdentifiersRequestBody'
        }
        response=issuePost(createInPortSchemaURL, assetManagerIdentifiersRequestBody)
       
        if response.status_code != 200:
            print ("No in port schema created")
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
            return []         
        
        # create out port
        outPortProperties = {
            'class':'PortProperties',
            'qualifiedName': processGuid + "-out-port",
            'portType': 'OUTPUT_PORT',
            'displayName': 'out port'
        }
        outPortRequestBody = {
            'class':'PortRequestBody',
            'MetadataCorrelationProperties': metadataCorrelationProperties,
            'elementProperties': outPortProperties
        } 

        response=issuePost(createPortURL, outPortRequestBody)
        outPortGuid = response.json().get('guid')
        if outPortGuid is None:
            print ("No out port created")
            processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)
        
        # create out portschema
        createOutPortSchemaURL = assetManagerURL + '/ports/' + outPortGuid + '/schema-type/' + targetSchemaTypeGuid +'?assetManagerIsHome=false'
        assetManagerIdentifiersRequestBody = {
            'class':'AssetManagerIdentifiersRequestBody'
        }
        response=issuePost(createOutPortSchemaURL, assetManagerIdentifiersRequestBody)
       
        if response.status_code != 200:
            print ("No out port schema created")
            processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)  
        else:
            print ("Process " + processName + " created.") 
        return processGuid
    else:
        print ("No process created")
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)
    

In [ ]:
#
# Subject Area OMAS
# Working with glossaries - this set of functions displays glossary content returned from the open metadata repositories.
#         

def printGlossarySummary(glossary):
    nodeTypeName = glossary.get('nodeType')
    nodeQualifiedName = glossary.get('qualifiedName')
    systemAttributes = glossary.get('systemAttributes')
    guid = systemAttributes.get('guid')
    print (nodeQualifiedName)
    print(nodeTypeName + " \t| " + guid + " | " + nodeQualifiedName)    
    
def printTermSummary(term):
    nodeTypeName = term.get('nodeType')
    nodeQualifiedName = term.get('qualifiedName')
    systemAttributes = term.get('systemAttributes')
    guid = systemAttributes.get('guid')
    print (nodeQualifiedName)
    print(nodeTypeName + " \t| " + guid + " | " + nodeQualifiedName)     

#
# Working with glossary content - this set of functions are for authoring glossary content.
#      
    
# Create a glossary with a display name and description     
def createGlossary(serverName, serverPlatformName, serverPlatformURL, userId, displayName, description):    
    subjectAreaURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/subject-area/users/' + userId
    createGlossaryURL = subjectAreaURL + '/glossaries'
    createGlossaryBody = {
        "class" : "Glossary",
        "name" : displayName,
        "description" : description,
        "nodeType" : "Glossary",
    }
    response = issuePost(createGlossaryURL, createGlossaryBody)
    result = response.json().get('result')
    createdGlossary = result[0]
    if createdGlossary:
        return createdGlossary
    else:
        print ("No Glossary created")
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)
        
# Create a glossary term with a display name and description     
def createTerm(serverName, serverPlatformName, serverPlatformURL, userId, glossaryGUID, displayName, description, example):
    subjectAreaURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/subject-area/users/' + userId
    createTermURL = subjectAreaURL + '/terms'
    
    glossaryBody = { 
        "guid": glossaryGUID
    }
    createTermBody = {
        "class" : "Term",
        "name" : displayName,
        "description" : description,
        "example"  : example,
        "nodeType" : "Term",
        "glossary" : glossaryBody
    }
 
    response = issuePost(createTermURL, createTermBody)
    result = response.json().get('result')
    createdTerm = result[0]
    if createdTerm:
        return createdTerm
    else:
        print ("No Term created")
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)
    
# Find a Term that matches the supplied search criteria and is within the glossary identified by glossaryGUID    
def findTerm(serverName, serverPlatformName, serverPlatformURL, userId, searchCriteria, glossaryGUID):
    subjectAreaURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/subject-area/users/' + userId
    termURL = subjectAreaURL + '/terms?searchCriteria=' + searchCriteria
    response=issueGet(termURL)
    results = response.json().get('result')
    for resultTerm in results:
        resultTermGlossary = resultTerm.get('glossary') 
        if (resultTermGlossary):
            resultTermGlossaryGUID = resultTermGlossary.get('guid')
            if resultTermGlossaryGUID == glossaryGUID:
                return resultTerm
            
    print ("No Term found")
    processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)

# update a Term with the supplied governance classifications     
def updateTermGovernanceClassifications(serverName, serverPlatformName, serverPlatformURL, userId, termGuid, retention, criticality, confidence, confidentiality):
    subjectAreaURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/subject-area/users/' + userId
    termURL = subjectAreaURL + '/terms/' + termGuid
    governanceClassifications = {
        "class": "GovernanceClassifications"
    }
    if retention:
        retention['class'] = "Retention"
        governanceClassifications['retention'] = retention
    if criticality:
        criticality['class'] = "Criticality"
        governanceClassifications['criticality'] = criticality
    if confidence:
        confidence['class'] = "Confidence"
        governanceClassifications['confidence'] = confidence
    if confidentiality:
        confidentiality['class'] = "confidentiality"
        governanceClassifications['confidentiality'] = confidentiality
        
    termBody = {
        "class" : "Term",
        "governanceClassifications": governanceClassifications
    }
    response=issuePut(termURL, termBody)
    result = response.json().get('result')

    updatedTerm = result[0]
    if updatedTerm:
        print("updatedTerm Governance action classifications")
        print(updatedTerm.get('governanceClassifications'))
        return updatedTerm
    else:
        print ("Term not updated")
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)
    
# create a synonym relationship between 2 glossary terms    
def createSynonym(serverName, serverPlatformName, serverPlatformURL, userId, guid1, guid2):
    subjectAreaURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/subject-area/users/' + userId
    createSynonymURL = subjectAreaURL + '/relationships/synonyms'
    
    end1 = {
        "class": "LineEnd",
        "nodeGuid" : guid1,
        "nodeType": "Term",
        "name":"synonyms"
    }
    end2 = {
        "class": "LineEnd",
        "nodeGuid" : guid2,
        "nodeType": "Term",
        "name":"synonyms"
    }
    createSynonymBody = {
        "class" : "Synonym",
        "typeDefGuid": "74f4094d-dba2-4ad9-874e-d422b69947e2",
        "lineType": "Synonym",
        "name": "Synonym",
        "end1": end1,
        "end2": end2
        
    }
    response=issuePost(createSynonymURL, createSynonymBody)
    result = response.json().get('result')
    createdSynonym = result[0]
    if createdSynonym:
        return createdSynonym
    else:
        print("No Synonym created")
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response)
    
# Create a valid value relationship between the valid value for and the valid value glossary terms.   
def createValidValue(serverName, serverPlatformName, serverPlatformURL, userId, validValueForGuid, validValuesGuid):
    subjectAreaURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/access-services/subject-area/users/' + userId
    createValidValueURL = subjectAreaURL + '/relationships/valid-values'

    end1 = {
        "class": "LineEnd",
        "nodeGuid" : validValueForGuid,
        "nodeType": "Term",
        "name":"validValueFor"
    }
    end2 = {
        "class": "LineEnd",
        "nodeGuid" : validValuesGuid,
        "nodeType": "Term",
        "name":"validValues"
    }
    createValidValuesBody = {
        "class" : "ValidValue",
        "typeDefGuid": "707a156b-e579-4482-89a5-de5889da1971",
        "lineType": "ValidValue",
        "name": "ValidValue",
        "end1": end1,
        "end2": end2   
    }

    response=issuePost(createValidValueURL, createValidValuesBody)
    result = response.json().get('result')
    createdValidValue = result[0]
    if createdValidValue:
        return createdValidValue
    else:
        print("No ValidValue relationship created")
        processErrorResponse(serverName, serverPlatformName, serverPlatformURL, response) 
        
# This function takes a glossary guid, and a file name. The file name should be a csv file that contains contain columns with names nameColumnHeader, descriptionColumnHeader, exampleColumn.
# - the fileName on the Mac platform that works is of the form /.../.../zzz.csv. The file needs to exist. The Python read might be sensitive to different file encodings.  
# - nameColumnHeader is the column for that will be used for the Glossary Term Name,
# - descriptionColumnHeader is the column for that will be used for the Glossary Term Description,
# - exampleColumn  is the column for that will be used for the Glossary Term Example
# 
# Note that  the qualifiedName for each term is not specified so it is generated by the Subject Area OMAS. 

def createSimpleTermsFromCSVFile(serverName, serverPlatformName, serverPlatformURL, userId, fileName, glossaryGUID, nameColumnHeader, descriptionColumnHeader, exampleColumnHeader):
    term_count =0 
    headers= []
    # open the file containing the glossary term definitions to get the headers
    try:
        with open(fileName, newline='', encoding='utf-8-sig') as csvfile:
            reader = csv.reader(csvfile)
            headers = next(reader)
    except EnvironmentError: # parent of IOError, OSError *and* WindowsError where available
        print("Error opening File to read headers" + fileName)    
    

    # open the file containing the glossary term definitions to read the data
    try:
        with open(fileName, newline='', encoding='utf-8-sig') as csvfile:
            isNameHeaderValid =False
            isDescriptionHeaderValid =False
            isExampleHeaderValid =False
            if (nameColumnHeader in headers):
                isNameHeaderValid =True
            if (descriptionColumnHeader in headers): 
                isDescriptionHeaderValid
            if (exampleColumnHeader in headers):       
                isExampleHeaderValid =True
            
            if (isNameHeaderValid):
                dictionary = csv.DictReader(csvfile)
                for dictionary_element in dictionary:
                    if (isDebug):
                        print(dictionary_element)
                    name_dictionary_element = dictionary_element[nameColumnHeader]   
                    if (name_dictionary_element == None or name_dictionary_element == ''):
                        print("Found a Row without a name - so ignoring.")
                    else:   
#                       only look for a description and example - if those rows exist in the csv file        
                        if (isDescriptionHeaderValid):      
                            description_dictionary_element = dictionary_element[descriptionColumnHeader]   
                        else:
                            description_dictionary_element = ""  
                        if (isExampleHeaderValid):
                            example_dictionary_element = dictionary_element[exampleColumnHeader]
                        else:
                            example_dictionary_element = ""     
                        createdTerm = createTerm(serverName, serverPlatformName, serverPlatformURL, userId, glossaryGUID, name_dictionary_element, 
                                 description_dictionary_element, example_dictionary_element )
                        if (isDebug):
                            printTermSummary(createdTerm)
                        term_count = term_count +1
            else:
                print("Expected Name column " + nameColumnHeader + " does not exist in the file, so we cannot process the file")              
    except EnvironmentError: # parent of IOError, OSError *and* WindowsError where available
        print("Error opening File " + fileName)         
    if (term_count > 0):        
        print("Created " + str(term_count))    
    else:
        print("No terms created")
        


In [ ]:
# Open Governance and Discovery
# Working with the engine host server and the Governance Engine OMAS of the metadata server.

## Governance Action Open Metadata Engine Services (OMES) validation of a governance service connector

def validateGovernanceActionEngineConnector(serverName, serverPlatformName, serverPlatformURL, userId, connectorProvider):
    try:
        engineServiceURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/engine-services/governance-action/users/' + userId 
        getStatusURL = engineServiceURL + '/validate-connector/' + connectorProvider
        response=issueGet(getStatusURL)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                connectorReport = response.json().get('connectorReport')
                if connectorReport:
                    connectorType = connectorReport.get('connectorType')
                    if connectorType:
                        print("   connectorType: ")
                        guid = connectorType.get('guid')
                        if guid:
                            print("       guid:                              " + guid)
                        qualifiedName = connectorType.get('qualifiedName')
                        if qualifiedName:
                            print("       qualifiedName:                     " + qualifiedName)
                        displayName = connectorType.get('displayName')
                        if displayName:
                            print("       displayName:                       " + displayName)
                        description = connectorType.get('description')
                        if description:
                            print("       description:                       " + description)
                        connectorProviderClassName = connectorType.get('connectorProviderClassName')
                        if connectorProviderClassName:
                            print("       connectorProviderClassName:        " + connectorProviderClassName)
                        if connectorType.get('connectorInterfaceLanguage'):
                            print ("    connectorInterfaceLanguage: " + connectorType.get('connectorInterfaceLanguage'))
                        if connectorType.get('connectorFrameworkName'):
                            print ("    connectorFrameworkName: " + connectorType.get('connectorFrameworkName'))
                        connectorInterfaces = connectorType.get('connectorInterfaces')
                        if connectorInterfaces:
                            print ("    connectorInterfaces:")
                            for x in range(len(connectorInterfaces)):
                                print ("        " + connectorInterfaces[x])
                        if connectorType.get('targetTechnologySource'):
                            print ("    targetTechnologySource: " + connectorType.get('targetTechnologySource'))
                        if connectorType.get('targetTechnologyName'):
                            print ("    targetTechnologyName: " + connectorType.get('targetTechnologyName'))
                        recognizedConfigurationProperties = connectorType.get('recognizedConfigurationProperties')
                        if recognizedConfigurationProperties:
                            print("       recognizedConfigurationProperties: ")
                            for x in range(len(recognizedConfigurationProperties)):
                                print("          - " + recognizedConfigurationProperties[x])
                        supportedRequestTypes = connectorReport.get('supportedRequestTypes')
                        if supportedRequestTypes:
                            print("   supportedRequestTypes: ")
                            for x in range(len(supportedRequestTypes)):
                                print("      - " + supportedRequestTypes[x])
                        supportedRequestParameters = connectorReport.get('supportedRequestParameters')
                        if supportedRequestParameters:
                            print("   supportedRequestParameters: ")
                            for x in range(len(supportedRequestParameters)):
                                print("      - " + supportedRequestParameters[x])
                        supportedRequestSourceNames = connectorReport.get('supportedRequestSourceNames')
                        if supportedRequestSourceNames:
                            print("   supportedRequestSourceNames: ")
                            for x in range(len(supportedRequestSourceNames)):
                                print("      - " + supportedRequestSourceNames[x])
                        supportedActionTargetNames = connectorReport.get('supportedActionTargetNames')
                        if supportedActionTargetNames:
                            print("   supportedActionTargetNames: ")
                            for x in range(len(supportedActionTargetNames)):
                                print("      - " + supportedActionTargetNames[x])
                        supportedGuards = connectorReport.get('supportedGuards')
                        if supportedGuards:
                            print("   supportedGuards: ")
                            for x in range(len(supportedGuards)):
                                print("      - " + supportedGuards[x])
                    else:
                        printResponse(response)
                else:
                    printResponse(response)
            else:
                printResponse(response)
        else:
            printResponse(response)
    except Exception as error:
        print("Exception: %s" % error)
        print("Platform " + serverPlatformURL + " is returning an error")

        
## Asset Analysis Open Metadata Engine Services (OMES) validation of a discovery service connector

def validateAssetAnalysisEngineConnector(serverName, serverPlatformName, serverPlatformURL, userId, connectorProvider):
    try:
        engineServiceURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/engine-services/asset-analysis/users/' + userId 
        getStatusURL = engineServiceURL + '/validate-connector/' + connectorProvider
        response=issueGet(getStatusURL)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                connectorReport = response.json().get('connectorReport')
                if connectorReport:
                    connectorType = connectorReport.get('connectorType')
                    if connectorType:
                        guid = connectorType.get('guid')
                        if guid:
                            print("   guid:                              " + guid)
                        qualifiedName = connectorType.get('qualifiedName')
                        if qualifiedName:
                            print("   qualifiedName:                     " + qualifiedName)
                        displayName = connectorType.get('displayName')
                        if displayName:
                            print("   displayName:                       " + displayName)
                        description = connectorType.get('description')
                        if description:
                            print("   description:                       " + description)
                        connectorProviderClassName = connectorType.get('connectorProviderClassName')
                        if connectorProviderClassName:
                            print("   connectorProviderClassName:        " + connectorProviderClassName)
                        if connectorType.get('connectorInterfaceLanguage'):
                            print ("    connectorInterfaceLanguage: " + connectorType.get('connectorInterfaceLanguage'))
                        if connectorType.get('connectorFrameworkName'):
                            print ("    connectorFrameworkName: " + connectorType.get('connectorFrameworkName'))
                        connectorInterfaces = connectorType.get('connectorInterfaces')
                        if connectorInterfaces:
                            print ("    connectorInterfaces:")
                            for x in range(len(connectorInterfaces)):
                                print ("        " + connectorInterfaces[x])
                        if connectorType.get('targetTechnologySource'):
                            print ("    targetTechnologySource: " + connectorType.get('targetTechnologySource'))
                        if connectorType.get('targetTechnologyName'):
                            print ("    targetTechnologyName: " + connectorType.get('targetTechnologyName'))
                        recognizedConfigurationProperties = connectorType.get('recognizedConfigurationProperties')
                        if recognizedConfigurationProperties:
                            print("")
                            print("   recognizedConfigurationProperties: ")
                            for x in range(len(recognizedConfigurationProperties)):
                                print("      - " + recognizedConfigurationProperties[x])
                    else:
                        printResponse(response)
                else:
                    printResponse(response)
            else:
                printResponse(response)
        else:
            printResponse(response)
    except Exception as error:
        print("Exception: %s" % error)
        print("Platform " + serverPlatformURL + " is returning an error")
        

# The configuration of governance engines and governance services is managed in an open metadata server
# through the Governance Engine OMAS.  The functions below support the definition and retrieval of these capabilities.

def getGovernanceEngineDefinitions(serverName, serverPlatformName, serverPlatformURL, userId):
    configCommandURLRoot = serverPlatformURL + "/servers/" + serverName + "/open-metadata/access-services/governance-engine/users/" + userId
    getGovernanceEngineURL = configCommandURLRoot + '/governance-engines?startingFrom=0&maximumResults=0'
    governanceEngineProperties = None
    response = requests.get(getGovernanceEngineURL, verify=False)
    if response.status_code == 200:
        relatedHTTPCode = response.json().get('relatedHTTPCode')
        if relatedHTTPCode == 200:
            governanceEngines = response.json().get('elements')
            if governanceEngines:
                for x in range(len(governanceEngines)):
                    governanceEngineGUID = printGovernanceEngineDefinition(governanceEngines[x])
                    if governanceEngineGUID:
                        getRegisteredGovernanceServices(serverName, serverPlatformName, serverPlatformURL, userId, governanceEngineGUID)
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    else:
        printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    return governanceEngineProperties

def printGovernanceEngineDefinition(governanceEngineElement):
    governanceEngineGUID = None
    governanceEngineHeader = governanceEngineElement.get('elementHeader')
    if governanceEngineHeader:
        governanceEngineTypeName = getElementType(governanceEngineHeader)
        governanceEngineGUID = getElementGUID(governanceEngineHeader)
    governanceEngineProperties = governanceEngineElement.get('properties')
    governanceEngineName = governanceEngineProperties.get('qualifiedName')
    if governanceEngineName:
        print(" ")
        print("  Governance Engine: " + governanceEngineName)
        if governanceEngineTypeName:
            print("     type:               " + governanceEngineTypeName)
        displayName = governanceEngineProperties.get('displayName')
        if displayName:
            print("     display name:       " + displayName)
        if governanceEngineHeader:
            print("     guid:               " + governanceEngineGUID)
        governanceEngineDescription = governanceEngineProperties.get('description')
        if governanceEngineDescription:
            print("     description:        " + governanceEngineDescription)
        print("     requestTypes:")
    return governanceEngineGUID
        

def getRegisteredGovernanceServices(serverName, serverPlatformName, serverPlatformURL, userId, governanceEngineGUID):
    configCommandURLRoot = serverPlatformURL + "/servers/" + serverName + "/open-metadata/access-services/governance-engine/users/" + userId
    getGovernanceEngineURL = configCommandURLRoot + '/governance-engines/' + governanceEngineGUID + '/governance-services?startingFrom=0&maximumResults=0'
    registeredServicesProperties = None
    response = requests.get(getGovernanceEngineURL, verify=False)
    if response.status_code == 200:
        relatedHTTPCode = response.json().get('relatedHTTPCode')
        if relatedHTTPCode == 200:
            registeredServicesProperties = response.json().get('elements')
            if registeredServicesProperties:
                for x in range(len(registeredServicesProperties)):
                    printRegisteredServicesProperties(registeredServicesProperties[x])
            else:
                print("            <none>")
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    else:
        printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    return registeredServicesProperties

def printRegisteredServicesProperties(registeredServiceProperties):
    governanceServiceHeader = registeredServiceProperties.get('elementHeader')
    if governanceServiceHeader:
        governanceServiceTypeName = getElementType(governanceServiceHeader)
        governanceServiceGUID     = getElementGUID(governanceServiceHeader)
    governanceServiceProperties   = registeredServiceProperties.get('properties')
    if governanceServiceProperties:
        governanceServiceDescription = governanceServiceProperties.get('description')
        requestTypes = governanceServiceProperties.get('requestTypes')
        if requestTypes:
            printRequestTypes("            ", 'requestTypes', requestTypes, governanceServiceDescription)

def printRequestTypes(indent, propertyName, requestTypes, governanceServiceDescription):
    if requestTypes:
        propertyList = list(requestTypes)
        for x in range(len(propertyList)):
            propertyValue = requestTypes.get(propertyList[x])
            if propertyValue:
                serviceRequestType = propertyValue.get('serviceRequestType')
                if serviceRequestType:
                    print(indent + propertyList[x] + " --> " + serviceRequestType + ":")
                else:
                    print(indent + propertyList[x] + ":")
            else:
                print(indent + propertyList[x] + ":" )
            descriptionWidth=80
            descriptionIndent = indent + "     "
            if governanceServiceDescription:
                description = governanceServiceDescription
                descriptionLength = len(description)
            else:
                description = ""
                descriptionLength = 0
            if descriptionLength <= descriptionWidth:
                print(descriptionIndent + description)
            else:
                textPointer = 0
                print(descriptionIndent  + description[textPointer:textPointer + descriptionWidth])
                textPointer = textPointer + descriptionWidth
                while textPointer < descriptionLength:
                    print(descriptionIndent + description[textPointer:textPointer + descriptionWidth])
                    textPointer = textPointer + descriptionWidth

    
def getGovernanceServiceDefinitions(serverName, serverPlatformName, serverPlatformURL, userId):
    configCommandURLRoot = serverPlatformURL + "/servers/" + serverName + "/open-metadata/access-services/governance-engine/users/" + userId
    getGovernanceServiceURL = configCommandURLRoot + '/governance-services?startingFrom=0&maximumResults=0'
    governanceServiceProperties = None
    response = requests.get(getGovernanceServiceURL, verify=False)
    if response.status_code == 200:
        relatedHTTPCode = response.json().get('relatedHTTPCode')
        if relatedHTTPCode == 200:
            governanceServices = response.json().get('elements')
            if governanceServices:
                for x in range(len(governanceServices)):
                    printGovernanceServiceDefinition(governanceServices[x])
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)                
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    else:
        printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    return governanceServiceProperties


def printGovernanceServiceDefinition(governanceServiceElement):
    governanceServiceHeader = governanceServiceElement.get('elementHeader')
    if governanceServiceHeader:
        governanceServiceTypeName = getElementType(governanceServiceHeader)
    governanceServiceProperties = governanceServiceElement.get('properties')
    governanceServiceName = governanceServiceProperties.get('qualifiedName')
    if governanceServiceName:
        print(" ")
        print("  Governance Service: " + governanceServiceName)
        if governanceServiceTypeName:
            print("     type:               " + governanceServiceTypeName)
        displayName = governanceServiceProperties.get('name')
        if displayName:
            print("     display name:       " + displayName)
        if governanceServiceHeader:
            print("     guid:               " + getElementGUID(governanceServiceHeader))
        governanceServiceDescription = governanceServiceProperties.get('description')
        if governanceServiceDescription:
            print("     description:        " + governanceServiceDescription)

def getElementType(elementHeader):
    if elementHeader:
        elementType = elementHeader.get('type')
        if elementType:
            return elementType.get('typeName')
    return None

def getElementGUID(elementHeader):
    if elementHeader:
        return elementHeader.get('guid')
    return None
   
    
def getGovernanceEngineGUID(serverName, serverPlatformName, serverPlatformURL, userId, qualifiedName):
    governanceEngineProperties = getGovernanceEngineProperties(serverName, serverPlatformName, serverPlatformURL, userId, qualifiedName)
    if governanceEngineProperties:
        elementHeader = governanceEngineProperties.get('elementHeader')
        if elementHeader:
            return elementHeader.get('guid')
    print ("Returns:")
    prettyResponse = json.dumps(governanceEngineProperties, indent=4)
    print (prettyResponse)
    print (" ")  


def getGovernanceEngineProperties(serverName, serverPlatformName, serverPlatformURL, userId, qualifiedName):
    configCommandURLRoot = serverPlatformURL + "/servers/" + serverName + "/open-metadata/access-services/governance-engine/users/" + userId
    getGovernanceEngineURL = configCommandURLRoot + '/governance-engines/by-name/' + qualifiedName
    governanceEngineProperties = None
    response = requests.get(getGovernanceEngineURL, verify=False)
    if response.status_code == 200:
        relatedHTTPCode = response.json().get('relatedHTTPCode')
        if relatedHTTPCode == 200:
            governanceEngineProperties = response.json().get('element')
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    else:
        printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    return governanceEngineProperties


def createGovernanceEngine(serverName, serverPlatformName, serverPlatformURL, userId, typeName, qualifiedName, displayName, description):
    governanceEngineProperties = getGovernanceEngineProperties(serverName, serverPlatformName, serverPlatformURL, userId, qualifiedName)
    if (governanceEngineProperties):
        print("Governance Engine " + qualifiedName + " is already defined with the following properties:")
        displayName = governanceEngineProperties.get('displayName')
        if (displayName):
            print("   displayName: " + displayName)
        description = governanceEngineProperties.get('description')
        if (description):
            print("   description: " + description)
        elementHeader = governanceEngineProperties.get('elementHeader')
        return elementHeader.get('guid')
    else:
        configCommandURLRoot = serverPlatformURL + "/servers/" + serverName + "/open-metadata/access-services/governance-engine/users/" + userId
        createGovernanceEngineURL = configCommandURLRoot + '/governance-engines/new/' + typeName
        governanceEngineGUID = None
        try:
            body = {
                "class" : "NewGovernanceEngineRequestBody",
                "qualifiedName" : qualifiedName,
                "displayName" : displayName,
                "description" : description
            }
            response=issuePost(createGovernanceEngineURL, body)
            if response.status_code == 200:
                relatedHTTPCode = response.json().get('relatedHTTPCode')
                if relatedHTTPCode == 200:
                    governanceEngineGUID = response.json().get('guid')
                else:
                    printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        except Exception as error:
            print("Exception: %s" % error)
            print("Platform " + serverPlatformURL + " is returning an error")
        return governanceEngineGUID


def createGovernanceService(serverName, serverPlatformName, serverPlatformURL, userId, typeName, qualifiedName, displayName, description, providerClassName, configurationProperties):
    configCommandURLRoot = serverPlatformURL + "/servers/" + serverName + "/open-metadata/access-services/governance-engine/users/" + userId        
    createGovernanceServiceURL = configCommandURLRoot + '/governance-services/new/' + typeName
    governanceServiceGUID = None
    try:
        body = {
            "class" : "NewGovernanceServiceRequestBody",
            "qualifiedName" : qualifiedName,
            "displayName" : displayName,
            "description" : description,
            "connection" : {
                "class": "Connection",
                "type": {
                    "class": "ElementType",
                    "elementTypeId": "114e9f8f-5ff3-4c32-bd37-a7eb42712253",
                    "elementTypeName": "Connection",
                    "elementTypeVersion": 1,
                    "elementTypeDescription": "A set of properties to identify and configure a connector instance.",
                    "elementOrigin": "CONFIGURATION"
                },
                "qualifiedName": qualifiedName + "-implementation",
                "displayName": displayName + " Implementation Connection",
                "description": "Connection for governance service " + qualifiedName,
                "connectorType": {
                    "class": "ConnectorType",
                    "type": {
                        "class": "ElementType",
                        "elementTypeId": "954421eb-33a6-462d-a8ca-b5709a1bd0d4",
                        "elementTypeName": "ConnectorType",
                        "elementTypeVersion": 1,
                        "elementTypeDescription": "A set of properties describing a type of connector.",
                        "elementOrigin": "LOCAL_COHORT"
                    },
                    "guid": "1111f73d-e343-abcd-82cb-3918fed81da6",
                    "qualifiedName": qualifiedName + "-GovernanceServiceProvider",
                    "displayName": displayName + " Governance Service Provider Implementation",
                    "description": description,
                    "connectorProviderClassName": providerClassName
                },
                "configurationProperties" : configurationProperties
            }
        }
        response=issuePost(createGovernanceServiceURL, body)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                governanceServiceGUID = response.json().get('guid')
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except Exception as error:
        print("Exception: %s" % error )
        print("Platform " + serverPlatformURL + " is returning an error")
    return governanceServiceGUID


def registerGovernanceServiceWithEngine(serverName, serverPlatformName, serverPlatformURL, userId, governanceEngineGUID, governanceServiceGUID, governanceRequestType, serviceRequestType):
    configCommandURLRoot = serverPlatformURL + "/servers/" + serverName + "/open-metadata/access-services/governance-engine/users/" + userId        
    registerGovernanceServiceURL = configCommandURLRoot + '/governance-engines/' + governanceEngineGUID + '/governance-services'
    governanceServerGUID = None
    try:
        body = {
            "class" : "GovernanceServiceRegistrationRequestBody",
            "governanceServiceGUID" : governanceServiceGUID,
            "requestType" : governanceRequestType,
            "serviceRequestType" : serviceRequestType
        }
        response=issuePost(registerGovernanceServiceURL, body)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                governanceServiceGUID = response.json().get('guid')
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except Exception as error:
        print("Exception: %s" % error )
        print("Platform " + serverPlatformURL + " is returning an error")


# The engine host periodically refreshes its configuration.  This call causes it to request fresh
# configuration from the Metadata Server immediately.  This is useful in demos or labs so that there is
# no need to wait for the asynchronous update if demonstrating governance engine configuration and you want
# any configuration changes to take effect immediately.

def refreshGovernanceEngineConfig(serverName, serverPlatformName, serverPlatformURL, userId, qualifiedName):
    governanceServerCommandURLRoot = serverPlatformURL + "/servers/" + serverName + "/open-metadata/engine-host-services/users/" + userId
    refreshConfigURL = governanceServerCommandURLRoot + '/governance-engines/' + qualifiedName + '/refresh-config'
    response = requests.get(refreshConfigURL, verify=False)
    if response.status_code == 200:
        relatedHTTPCode = response.json().get('relatedHTTPCode')
        if relatedHTTPCode == 200:
            print("Done.")
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    else:
        printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)

        
# The engine can return the current status of the governance engines that it is hosting.
# This is useful to show how the Engine Host is configured and the different types of
# governance requests that are supported.

def printGovernanceEngineStatus(governanceEngineSummary):
    governanceEngineName = governanceEngineSummary.get('governanceEngineName')
    if governanceEngineName:
        print("  Governance Engine: " + governanceEngineName)
        governanceEngineTypeName = governanceEngineSummary.get('governanceEngineTypeName')
        if governanceEngineTypeName:
            print("     type:               " + governanceEngineTypeName)
        governanceEngineService = governanceEngineSummary.get('governanceEngineService')
        if governanceEngineService:
            print("     supporting service: " + governanceEngineService)
        governanceEngineStatus = governanceEngineSummary.get('governanceEngineStatus')
        if governanceEngineStatus:
            print("     status:             " + governanceEngineStatus)    
        governanceEngineGUID = governanceEngineSummary.get('governanceEngineGUID')
        if governanceEngineGUID:
            print("     guid:               " + governanceEngineGUID)
        governanceEngineDescription = governanceEngineSummary.get('governanceEngineDescription')
        if governanceEngineDescription:
            print("     description:        " + governanceEngineDescription)
        requestTypes = governanceEngineSummary.get('governanceRequestTypes')
        if requestTypes:
            print("     requestTypes:       ")
            for x in range(len(requestTypes)):
                print("       - " + requestTypes[x])

def checkGovernanceEngineStatus(governanceEngineSummary, desiredStatus):
    governanceEngineStatus = governanceEngineSummary.get('governanceEngineStatus')
    if governanceEngineStatus == desiredStatus:
        return True
    else:
        return False

def getGovernanceEngineSummary(serverName, serverPlatformName, serverPlatformURL, userId, governanceEngineName):
    governanceEngineSummary = None
    try:
        governanceServerRootURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/engine-host-services/users/' + userId
        getStatusURL = governanceServerRootURL + '/governance-engines/' + governanceEngineName + '/summary'
        response = requests.get(getStatusURL, verify=False)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                governanceEngineSummary = response.json().get('governanceEngineSummary')
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except Exception as error:
        print("Exception: %s" % error)
        print("Platform " + serverPlatformURL + " is returning an error")
    return governanceEngineSummary
    
def waitForConfiguringGovernanceEngine(serverName, serverPlatformName, serverPlatformURL, userId, governanceEngineName):
    waitForGovernanceEngineStatus(serverName, serverPlatformName, serverPlatformURL, userId, governanceEngineName, "CONFIGURING")
        
def waitForRunningGovernanceEngine(serverName, serverPlatformName, serverPlatformURL, userId, governanceEngineName):
    waitForGovernanceEngineStatus(serverName, serverPlatformName, serverPlatformURL, userId, governanceEngineName, "RUNNING")

def waitForGovernanceEngineStatus(serverName, serverPlatformName, serverPlatformURL, userId, governanceEngineName, desiredStatus):
    time.sleep(1)
    governanceEngineSummary = getGovernanceEngineSummary(serverName, serverPlatformName, serverPlatformURL, userId, governanceEngineName)
    if governanceEngineSummary != None: 
        while not checkGovernanceEngineStatus(governanceEngineSummary, desiredStatus):
            refreshGovernanceEngineConfig(serverName, serverPlatformName, serverPlatformURL, userId, governanceEngineName)
            print(" ... Waiting for status change")
            time.sleep(1)
            governanceEngineSummary = getGovernanceEngineSummary(serverName, serverPlatformName, serverPlatformURL, userId, governanceEngineName)
        print("")

def printGovernanceEngineStatuses(serverName, serverPlatformName, serverPlatformURL, userId):
    try:
        governanceServerRootURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/engine-host-services/users/' + userId
        getStatusURL = governanceServerRootURL + '/governance-engines/summary'
        response = requests.get(getStatusURL, verify=False)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                governanceEngineSummaries = response.json().get('governanceEngineSummaries')
                if governanceEngineSummaries:
                    if len(governanceEngineSummaries) == 1:
                        print("One governance engine defined for engine host server " + serverName)
                    else:
                        print(str(len(governanceEngineSummaries)) + " governance engines defined for engine host server " + serverName)
                    print(' ')
                    for x in range(len(governanceEngineSummaries)):
                        printGovernanceEngineStatus(governanceEngineSummaries[x])
                        print(" ")
                    time.sleep(1)
                else:
                    print("No governance engines defined for engine host server " + serverName) 
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except Exception as error:
        print("Exception: %s" % error)
        print("Platform " + serverPlatformURL + " is returning an error")


def runDiscoveryService(serverName, serverPlatformName, serverPlatformURL, userId, discoveryEngineName, discoveryRequestType, requestType):
    try:
        discoveryServerRootURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/engine-services/asset-analysis/users/' + userId
        requestDiscoveryURL = discoveryServerRootURL + '/discovery-engines/' + discoveryEngineName + '/discovery-request-types/' + discoveryRequestType + '/assets/' + requestType
        body = {
            "class" : "DiscoveryRequestRequestBody",
        }
        response=issuePost(requestDiscoveryURL, body)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                reportGUID = response.json().get('guid')
                if reportGUID:
                    return reportGUID
                else:
                    print("No discovery results returned from Asset Analysis OMES in server " + serverName) 
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except Exception as error:
        print("Exception: %s" % error)
        print("Platform " + serverPlatformURL + " is returning an error")


def getDiscoveryReport(serverName, serverPlatformName, serverPlatformURL, userId, discoveryEngineName, reportGUID):
    try:
        discoveryServerRootURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/engine-services/asset-analysis/users/' + userId
        requestReportURL = discoveryServerRootURL + '/discovery-engines/' + discoveryEngineName + '/discovery-analysis-reports/' + reportGUID
        response=issueGet(requestReportURL)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                printResponse(response)
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except Exception as error:
        print("Exception: %s" % error)
        print("Platform " + serverPlatformURL + " is returning an error")
        
def getDiscoveryReportAnnotations(serverName, serverPlatformName, serverPlatformURL, userId, discoveryEngineName, reportGUID):
    try:
        discoveryServerRootURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/engine-services/asset-analysis/users/' + userId
        requestReportURL = discoveryServerRootURL + '/discovery-engines/' + discoveryEngineName + '/discovery-analysis-reports/' + reportGUID + '/annotations?startingFrom=0&maximumResults=100'
        response=requests.get(requestReportURL, verify=False)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                printResponse(response)
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except Exception as error:
        print("Exception: %s" % error)
        print("Platform " + serverPlatformURL + " is returning an error")


In [ ]:
#
# Working with Governance Actions, Governance Action Processes and Governance Action Types through the Governance Engine OMAS
#

def initiateGovernanceAction(serverName, serverPlatformName, serverPlatformURL, userId, governanceEngineName, qualifiedName, requestType, requestParameters, processName):
    commandURLRoot = serverPlatformURL + "/servers/" + serverName + "/open-metadata/framework-services/governance-engine/open-governance-service/users/" + userId        
    initiateGovernanceActionURL = commandURLRoot + '/governance-engines/' + governanceEngineName + '/governance-actions/initiate'
    governanceActionGUID = None
    try:
        body = {
            "class" : "GovernanceActionRequestBody",
            "qualifiedName" : qualifiedName,
            "processName" : processName,
            "requestType" : requestType,
            "requestParameters" : requestParameters
        }
        response=issuePost(initiateGovernanceActionURL, body)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                governanceActionGUID = response.json().get('guid')
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except Exception as error:
        print("Exception: %s" % error )
        print("Platform " + serverPlatformURL + " is returning an error")
    return governanceActionGUID
     
def printGovernanceAction(serverName, serverPlatformName, serverPlatformURL, userId, governanceActionGUID):
    commandURLRoot = serverPlatformURL + "/servers/" + serverName + "/open-metadata/access-services/governance-engine/users/" + userId        
    getGovernanceActionURL = commandURLRoot + '/governance-actions/' + governanceActionGUID
    try:
        response=issueGet(getGovernanceActionURL)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                element = response.json().get('element')
                if element:
                    qualifiedName = element.get('qualifiedName')
                    elementHeader = element.get('elementHeader')
                    if elementHeader:
                        guid = elementHeader.get('guid')
                    governanceEngineName = element.get('governanceEngineName')
                    processName = element.get('processName')
                    governanceRequestType = element.get('requestType')
                    requestParameters = element.get('requestParameters')
                    actionStatus = element.get('actionStatus')
                    requestedTime = element.get('requestedTime')
                    startTime = element.get('startTime')
                    processingEngineUserId = element.get('processingEngineUserId')
                    completionTime = element.get('completionTime')
                    completionGuards = element.get('completionGuards')
                    completionMessage = element.get('completionMessage')
                    print(qualifiedName + " [" + guid + "]")
                    if processName:
                        print("    processName:            " + processName)
                    if governanceEngineName:
                        print("    governanceEngineName:   " + governanceEngineName)
                    if governanceRequestType:
                        print("    requestType:            " + governanceRequestType)
                    if requestParameters:
                        printMapProperties("    ", "requestParameters", requestParameters)
                    if processingEngineUserId:
                        print("    processingEngineUserId: " + processingEngineUserId)
                    if actionStatus:
                        print("    actionStatus:           " + actionStatus)
                    if startTime:
                        print("    startTime:              " + startTime)
                    if requestedTime:
                        print("    requestedTime:          " + requestedTime)
                    if completionTime:
                        print("    completionTime:         " + completionTime)
                    if completionGuards:
                        print("     completionGuards:     ")
                        for x in range(len(completionGuards)):
                            print("         - " + completionGuards[x])
                    if completionMessage:
                        print("    completionMessage:      " + completionMessage)
                else:
                    prettyResponse = json.dumps(response, indent=4)
                    print(prettyResponse)
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except Exception as error:
        print("Exception: %s" % error )
        print("Platform " + serverPlatformURL + " is returning an error")

def checkGovernanceActionStatus(governanceActionElement, desiredState):
    governanceActionStatus = governanceActionElement.get('actionStatus')
    if governanceActionStatus == desiredState:
        return True
    else:
        return False
    
def checkGovernanceActionCompletion(governanceActionElement):
    governanceActionCompletionTime = None
    governanceActionCompletionTime = governanceActionElement.get('completionTime')
    if governanceActionCompletionTime:
        return True
    else:
        return False

def getGovernanceAction(serverName, serverPlatformName, serverPlatformURL, userId, governanceActionGUID):
    governanceActionElement = None
    try:
        commandURLRoot = serverPlatformURL + "/servers/" + serverName + "/open-metadata/access-services/governance-engine/users/" + userId        
        getGovernanceActionURL = commandURLRoot + '/governance-actions/' + governanceActionGUID
        response = requests.get(getGovernanceActionURL, verify=False)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                governanceActionElement = response.json().get('element')
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except Exception as error:
        print("Exception: %s" % error)
        print("Platform " + serverPlatformURL + " is returning an error")
    return governanceActionElement
    
def waitForRunningGovernanceAction(serverName, serverPlatformName, serverPlatformURL, userId, governanceActionGUID):
    governanceActionElement = getGovernanceAction(serverName, serverPlatformName, serverPlatformURL, userId, governanceActionGUID)
    if governanceActionElement != None: 
        while not checkGovernanceActionCompletion(governanceActionElement):
            governanceActionStatus = governanceActionElement.get('actionStatus')
            print(" ... Waiting for completion status, current status is: " + governanceActionStatus)
            time.sleep(1)
            governanceActionElement = getGovernanceAction(serverName, serverPlatformName, serverPlatformURL, userId, governanceActionGUID)
        print("")

def getGovernanceActions(serverName, serverPlatformName, serverPlatformURL, userId):
    governanceActions = None
    try:
        commandURLRoot = serverPlatformURL + "/servers/" + serverName + "/open-metadata/access-services/governance-engine/users/" + userId        
        getGovernanceActionURL = commandURLRoot + '/governance-actions?startFrom=0&pageSize=0'
        response = requests.get(getGovernanceActionURL, verify=False)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                governanceActions = response.json().get('elements')
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except Exception as error:
        print("Exception: %s" % error)
        print("Platform " + serverPlatformURL + " is returning an error")
    return governanceActions

def printGovernanceActionSummary(governanceAction):
    if governanceAction:
        name = governanceAction.get('displayName')
        if not name:
            name = governanceAction.get('qualifiedName')
        actionStatus = governanceAction.get('actionStatus')
        if governanceAction.get('completionGuards'):
            completionGuards = governanceAction.get('completionGuards')
        else:
            completionGuards = "\t"
        if governanceAction.get('processName'):
            processName = governanceAction.get('processName')
        else:
            processName = "\t"
        if governanceAction.get('completionMessage'):
            completionMessage = governanceAction.get('completionMessage')
        else:
            completionMessage = ""
        print(actionStatus + "\t| " + name + "\t| " + processName + "\t| " + '%s' % ', '.join(map(str, completionGuards)) + "\t| " + completionMessage)

def monitorGovernanceActions(serverName, serverPlatformName, serverPlatformURL, userId):
    governanceActions = getGovernanceActions(serverName, serverPlatformName, serverPlatformURL, userId)
    if governanceActions != None:
        for x in range(len(governanceActions)):
            printGovernanceActionSummary(governanceActions[x])

def createGovernanceActionProcess(serverName, serverPlatformName, serverPlatformURL, userId, qualifiedName, displayName, description, technicalName, technicalDescription):
    commandURLRoot = serverPlatformURL + "/servers/" + serverName + "/open-metadata/access-services/governance-engine/users/" + userId        
    createGovernanceActionProcessURL = commandURLRoot + '/governance-action-processes/new'
    governanceActionProcessGUID = None
    try:
        body = {
            "class" : "NewGovernanceActionProcessRequestBody",
            "processStatus" : "ACTIVE",
            "properties" : {
                "class" : "GovernanceActionProcessProperties",
                "qualifiedName" : qualifiedName,
                "displayName" : displayName,
                "description" : description,
                "technicalName" : technicalName,
                "technicalDescription" : technicalDescription,
                "owner" : userId,
                "ownerCategory" : "USER_ID"
            }
        }
        response=issuePost(createGovernanceActionProcessURL, body)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                governanceActionProcessGUID = response.json().get('guid')
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except Exception as error:
        print("Exception: %s" % error )
        print("Platform " + serverPlatformURL + " is returning an error")
    return governanceActionProcessGUID


def createGovernanceActionType(serverName, serverPlatformName, serverPlatformURL, userId, governanceEngineGUID, qualifiedName, supportedGuards, requestType, requestParameters):
    commandURLRoot = serverPlatformURL + "/servers/" + serverName + "/open-metadata/access-services/governance-engine/users/" + userId        
    createGovernanceActionTypeURL = commandURLRoot + '/governance-action-types/new'
    governanceActionTypeGUID = None
    try:
        body = {
            "class" : "GovernanceActionTypeProperties",
            "qualifiedName" : qualifiedName,
            "supportedGuards" : supportedGuards,
            "governanceEngineGUID" : governanceEngineGUID,
            "requestType" : requestType,
            "requestParameters" : requestParameters
        }
        response=issuePost(createGovernanceActionTypeURL, body)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                governanceActionTypeGUID = response.json().get('guid')
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except Exception as error:
        print("Exception: %s" % error )
        print("Platform " + serverPlatformURL + " is returning an error")
    return governanceActionTypeGUID

def setupFirstActionType(serverName, serverPlatformName, serverPlatformURL, userId, processGUID, actionTypeGUID, optionalGuard):
    commandURLRoot = serverPlatformURL + "/servers/" + serverName + "/open-metadata/access-services/governance-engine/users/" + userId        
    firstActionTypeURL = commandURLRoot + '/governance-action-processes/' + processGUID + '/first-action-type/' + actionTypeGUID + '/new'
    try:
        body = optionalGuard
        response=issueDataPost(firstActionTypeURL, body)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode != 200:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except Exception as error:
        print("Exception: %s" % error )
        print("Platform " + serverPlatformURL + " is returning an error")
        
def setupNextActionType(serverName, serverPlatformName, serverPlatformURL, userId, currentActionTypeGUID, nextActionTypeGUID, guard, mandatoryGuard, ignoreMultipleTriggers):
    commandURLRoot = serverPlatformURL + "/servers/" + serverName + "/open-metadata/access-services/governance-engine/users/" + userId        
    nextActionTypeURL = commandURLRoot + '/governance-action-types/' + currentActionTypeGUID + '/next-action-types/' + nextActionTypeGUID + '/new'
    try:
        body = {
            "class" : "NewGovernanceActionTypeRequestBody",
            "guard" : guard,
            "mandatoryGuard" : mandatoryGuard,
            "ignoreMultipleTriggers" : ignoreMultipleTriggers
        }
        response=issuePost(nextActionTypeURL, body)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode != 200:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except Exception as error:
        print("Exception: %s" % error )
        print("Platform " + serverPlatformURL + " is returning an error")


In [ ]:
#
# Using the OpenMetadataStore services to report on running governance action processes.
#



In [ ]:
# 
# Interaction with the Integration Daemon OMAG Server
#
def printIntegrationConnectorStatus(integrationConnectorSummary):
    connectorName = integrationConnectorSummary.get('connectorName')
    if connectorName:
        print("      Integration Connector: " + connectorName)
        connectorStatus = integrationConnectorSummary.get('connectorStatus')
        if connectorStatus:
            print("         status:                   " + connectorStatus)
            if connectorStatus == "FAILED":
                failingExceptionMessage = integrationConnectorSummary.get('failingExceptionMessage')
                if failingExceptionMessage:
                    print("         failingExceptionMessage:  " + failingExceptionMessage)
            else:
                lastStatusChange = integrationConnectorSummary.get('lastStatusChange')
                if lastStatusChange:
                    print("         lastStatusChange:         " + lastStatusChange)
                lastRefreshTime = integrationConnectorSummary.get('lastRefreshTime')
                if lastRefreshTime:
                    print("         lastRefreshTime:          " + lastRefreshTime)
                minMinutesBetweenRefresh = integrationConnectorSummary.get('minMinutesBetweenRefresh')
                if minMinutesBetweenRefresh:
                    print("         minMinutesBetweenRefresh: " + str(minMinutesBetweenRefresh))

def printIntegrationServiceSummary(integrationServiceSummary):
    integrationServiceName = integrationServiceSummary.get('integrationServiceFullName')
    if integrationServiceName:
        print("Integration Service: " + integrationServiceName)
        connectors = integrationServiceSummary.get('integrationConnectorReports')
        if connectors:
            print("  Integration Connector Reports: ")
            for x in range(len(connectors)):
                printIntegrationConnectorStatus(connectors[x])
                
def printIntegrationGroupSummary(integrationGroupSummary):
    integrationGroupName = integrationServiceSummary.get('integrationGroupName')
    if integrationGroupName:
        print("Integration Group: " + integrationGroupName)
        connectors = integrationGroupSummary.get('integrationConnectorReports')
        if connectors:
            print("  Integration Connector Reports: ")
            for x in range(len(connectors)):
                printIntegrationConnectorStatus(connectors[x])

def getIntegrationDaemonStatus(serverName, serverPlatformName, serverPlatformURL, userId):
    try:
        integrationDaemonURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/integration-daemon/users/' + userId 
        getStatusURL = integrationDaemonURL + '/status'
        response=issueGet(getStatusURL)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                integrationDaemonStatus = response.json().get('integrationDaemonStatus')
                if integrationDaemonStatus:
                    integrationServiceSummaries = integrationDaemonStatus.get('integrationServiceSummaries')
                    if integrationServiceSummaries:
                        if len(integrationServiceSummaries) == 1:
                            print("One integration service defined for integration daemon " + serverName)
                        else:
                            print(str(len(integrationServiceSummaries)) + " integration services defined for integration daemon " + serverName)
                        print(' ')
                        for x in range(len(integrationServiceSummaries)):
                            printIntegrationServiceSummary(integrationServiceSummaries[x])
                            print(" ")
                    else:
                        print("No integration services defined for integration daemon " + serverName)
                    integrationGroupSummaries = integrationDaemonStatus.get('integrationGroupSummaries')
                    if integrationGroupSummaries:
                        if len(integrationGroupSummaries) == 1:
                            print("One integration group defined for integration daemon " + serverName)
                        else:
                            print(str(len(integrationGroupSummaries)) + " integration groups defined for integration daemon " + serverName)
                        print(' ')
                        for x in range(len(integrationGroupSummaries)):
                            printIntegrationGroupSummary(integrationGroupSummaries[x])
                            print(" ")
                    else:
                        print("No integration group defined for integration daemon " + serverName)
                else:
                    print("No status returned for integration daemon " + serverName) 
            else:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except Exception as error:
        print("Exception: %s" % error)
        print("Platform " + serverPlatformURL + " is returning an error")

def restartIntegrationConnector(serverName, serverPlatformName, serverPlatformURL, userId, serviceURLMarker, connectorName):
    try:
        integrationDaemonURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/integration-daemon/users/' + userId 
        restartURL = integrationDaemonURL + '/integration-services/' + serviceURLMarker + '/restart'
        body = {
            "class" : "NameRequestBody",
            "name" : connectorName
        }
        response=issuePost(restartURL, body)
        if response.status_code == 200:
            print("Done.") 
        else:
            printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
    except Exception as error:
        print("Exception: %s" % error)
        print("Platform " + serverPlatformURL + " is returning an error")

def updateConnectorConfigurationProperties(serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, connectorName, configurationProperties):
    try:
        integrationDaemonURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/integration-daemon/users/' + userId + '/integration-services/' + serviceURLMarker 
        updateConnectorURL = integrationDaemonURL + '/connectors/configuration-properties'
        body = {
            "class" : "ConnectorConfigPropertiesRequestBody",
            "connectorName" : connectorName,
            "mergeUpdate" : True,
            "configurationProperties" : configurationProperties
        }
        response=issuePost(updateConnectorURL, body)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode != 200:
                printUnexpectedResponse(serverName, serverPlatformName, serverPlatformURL, response)
            else:
                print("Done.")
        else:
            printResponse(response)
    except Exception as error:
        print("Exception: %s" % error)
        print("Platform " + serverPlatformURL + " is returning an error") 
        
def getIntegrationConnectorConfigProperties(serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, connectorName):
    try:
        integrationDaemonURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/integration-daemon/users/' + userId + '/integration-services/' + serviceURLMarker 
        getConfigPropertiesURL = integrationDaemonURL + '/connectors/' + connectorName + '/configuration-properties'
        response=issueGet(getConfigPropertiesURL)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                printResponse(response)
            else:
                printResponse(response)
        else:
            printResponse(response)
    except Exception as error:
        print("Exception: %s" % error)
        print("Platform " + serverPlatformURL + " is returning an error")
        

In [ ]:
## Open Metadata Integration Services

def validateIntegrationConnector(serverName, serverPlatformName, serverPlatformURL, serviceURLMarker, userId, connectorProvider):
    try:
        integrationServiceURL = serverPlatformURL + '/servers/' + serverName + '/open-metadata/integration-services/' + serviceURLMarker + '/users/' + userId 
        getStatusURL = integrationServiceURL + '/validate-connector/' + connectorProvider
        response=issueGet(getStatusURL)
        if response.status_code == 200:
            relatedHTTPCode = response.json().get('relatedHTTPCode')
            if relatedHTTPCode == 200:
                connectorReport = response.json().get('connectorReport')
                connectorType = connectorReport.get('connectorType')
                if connectorType:
                    guid = connectorType.get('guid')
                    if guid:
                        print("   guid:                              " + guid)
                    qualifiedName = connectorType.get('qualifiedName')
                    if qualifiedName:
                        print("   qualifiedName:                     " + qualifiedName)
                    displayName = connectorType.get('displayName')
                    if displayName:
                        print("   displayName:                       " + displayName)
                    description = connectorType.get('description')
                    if description:
                        print("   description:                       " + description)
                    connectorProviderClassName = connectorType.get('connectorProviderClassName')
                    if connectorProviderClassName:
                        print("   connectorProviderClassName:        " + connectorProviderClassName)
                    recognizedConfigurationProperties = connectorType.get('recognizedConfigurationProperties')
                    if recognizedConfigurationProperties:
                        print("")
                        print("   recognizedConfigurationProperties: ")
                        for x in range(len(recognizedConfigurationProperties)):
                            print("      - " + recognizedConfigurationProperties[x])
                    if connectorType.get('connectorInterfaceLanguage'):
                        print ("    connectorInterfaceLanguage: " + connectorType.get('connectorInterfaceLanguage'))
                        if connectorType.get('connectorFrameworkName'):
                            print ("    connectorFrameworkName: " + connectorType.get('connectorFrameworkName'))
                            connectorInterfaces = connectorType.get('connectorInterfaces')
                            if connectorInterfaces:
                                print ("    connectorInterfaces:")
                                for x in range(len(connectorInterfaces)):
                                    print ("        " + connectorInterfaces[x])
                    if connectorType.get('targetTechnologySource'):
                        print ("    targetTechnologySource: " + connectorType.get('targetTechnologySource'))
                    if connectorType.get('targetTechnologyName'):
                        print ("    targetTechnologyName: " + connectorType.get('targetTechnologyName'))
                    refreshTimeInterval = connectorReport.get('refreshTimeInterval')
                    usesBlockingCalls = connectorReport.get('usesBlockingCalls')
                    if usesBlockingCalls:
                        print ("    usesBlockingCalls: true")
                    else:
                        print ("    usesBlockingCalls: false")
                    if refreshTimeInterval == 0:
                        print ("    refreshTimeInterval: no recommendation")
                    else:
                        print ("    refreshTimeInterval: " + str(refreshTimeInterval))
                else:
                    printResponse(response)
            else:
                printResponse(response)
        else:
            printResponse(response)
    except Exception as error:
        print("Exception: %s" % error)
        print("Platform " + serverPlatformURL + " is returning an error")
        
